In [36]:
import os
import numpy as np
import pandas as pd

In [37]:
def find_project_root(marker='.git'):
    """
    从当前目录向上搜索，直到找到包含指定标记目录的父目录，将其视为项目根目录。

    Args:
        marker: 用于标识项目根目录的子目录名称列表，默认为 ['.git', '.idea']。
    Returns:
        str: 项目根目录的绝对路径，如果未找到标记目录，则返回 None。
    """
    # 获取当前脚本或 Notebook 的工作目录
    # 在 Jupyter Notebook 中，os.getcwd() 通常是 Notebook 文件所在的目录
    current_dir = os.path.abspath(os.getcwd())

    while True:
        # 检查当前目录是否包含任何一个标记目录
        if os.path.exists(os.path.join(current_dir, marker)):
            return current_dir  # 找到根目录，返回

        # 向上移动到父目录
        parent_dir = os.path.dirname(current_dir)
        current_dir = parent_dir


project_root_path = find_project_root()

file_names = ['附件1_处理_final.xlsx', '附件2_处理_final.xlsx', '附件3_处理_final.xlsx']
data_subdirectory = '处理后的数据'
file_path = [os.path.join(project_root_path, data_subdirectory, name) for name in file_names]

In [38]:
df_man_origin = pd.read_excel(file_path[0])
df_woman_origin = pd.read_excel(file_path[1])

df_man = df_man_origin.copy()
df_woman = df_woman_origin.copy()

df_man.name = '男'
df_woman.name = '女'

In [39]:
def clean_food_code(code):
    """
    标准化食物编码：。
    1. 将编码转换为字符串并去除首尾空白。
    2. 分离出可能的末尾 'x' 和前面的主体部分。
    3. 如果是纯数字，根据长度（<6）决定是否补零。
    4. 将处理好的数字部分与末尾的 'x' 重新组合。
    5. 如果主体部分不全由数字组成，返回原始字符串。
    """
    # 1. 将编码转换为字符串并去除首尾空白
    code_str = str(code).strip()

    # 2. 分离出可能的末尾 'x' 和前面的主体部分
    numeric_part = code_str
    suffix_x = ''
    # 检查是否以 'x' 结尾（不区分大小写）
    if code_str.lower().endswith('x'):
        numeric_part = code_str[:-1]  # 取除最后一个字符外的所有部分作为数字主体
        suffix_x = code_str[-1]  # 获取最后一个字符，即 'x' 或 'X'

    # 3. 根据长度决定是否补零
    if len(numeric_part) < 6:
        # 长度小于6，补零至6位
        padded_numeric_part = numeric_part.zfill(6)
        # 6. 将处理好的数字部分与末尾的 'x' 重新组合
        return padded_numeric_part + suffix_x
    else:
        # 长度等于或大于6（且假设没有 >6 情况，或大于等于6都不补零）
        # 直接返回原字符串，因为数字主体已经是6位或更长，且末尾的 x 也已包含在原字符串中
        return code_str


def clean_food_code_main(df):
    df['食物编码'] = df['食物编码'].apply(clean_food_code)


clean_food_code_main(df_man)
clean_food_code_main(df_woman)

## 1．分析食物结构
代码


按类别将食谱中食物归类排序，
- 列出每种食物的数量
- 分析五大类别食物是否齐全

（1）谷、薯类；01,02

（2）蔬菜、菌藻、水果类； 04,05,06

（3）畜、禽、鱼、蛋类及制品；08,09,12,11

（4）奶、干豆、坚果、种子类及制品；10, ,07,

（5）植物油类。

- 食物种类是否大于12种
- 周食谱评价要求大于25种

In [40]:
five_major_food_names = ["谷、薯类", "蔬菜、菌藻、水果类", "畜、禽、鱼、蛋类及制品", "奶、干豆、坚果、种子类及制品", "植物油类"]

five_major_food_groups = [
    {"谷类及制品": "01", "薯类、淀粉及制品": "02"},
    {"蔬菜类及制品": "04", "菌藻类": "05", "水果类及制品": "06"},
    {"畜肉类及制品": "08", "禽肉类及制品": "09", "鱼虾蟹贝类": "12", "蛋类及制品": "11"},
    {"乳类及制品": "10", "干豆类及制品": "03", "坚果、种子类": "07"},
    {"植物油类": "18"}
]
# 建立编号和组名的映射
code_to_group_map = {}
for i, group_codes_dict in enumerate(five_major_food_groups):
    group_name = five_major_food_names[i]

    for _, code in group_codes_dict.items():
        code_to_group_map[code] = group_name
code_to_group_map


{'01': '谷、薯类',
 '02': '谷、薯类',
 '04': '蔬菜、菌藻、水果类',
 '05': '蔬菜、菌藻、水果类',
 '06': '蔬菜、菌藻、水果类',
 '08': '畜、禽、鱼、蛋类及制品',
 '09': '畜、禽、鱼、蛋类及制品',
 '12': '畜、禽、鱼、蛋类及制品',
 '11': '畜、禽、鱼、蛋类及制品',
 '10': '奶、干豆、坚果、种子类及制品',
 '03': '奶、干豆、坚果、种子类及制品',
 '07': '奶、干豆、坚果、种子类及制品',
 '18': '植物油类'}

代码思路:

读取男女生的表,

按照食物编码进行统计数量, 怎么得出每一种的数量?

将男女生的食物编码进行拆分, 保留前两个数字, 设为新列, 即为类别编码,



### 1.1 统计食物数量, 即每种食物的克重

In [41]:
# 计算每个食物的总克重
def calculate_total_grams(df):
    """
    :param df: 男/女生数据的Dataframe
    :return: 返回各食物的数量
    """
    print("--- 1.1 统计食物数量, 即每种食物的克重 ---")
    df['食物重量(克)'] = df['可食部（克/份）'] * df['食用份数']
    food_quantities_summary = df.groupby('食物名称')['食物重量(克)'].sum().reset_index()
    print(food_quantities_summary)
    return food_quantities_summary


### 1.2 分析五大类别食物是否齐全

In [42]:
def get_food_categories(df):
    # 定义一个函数，根据食物编码获取其所属的五大类别名称或标记为“其他类别”
    def get_major_food_group_from_code(food_code, code_to_group_map):
        """
        根据食物编码的前两位查找对应的五大类别名称。
        非五大类别，则返回 '其他类别'。
        """
        code = food_code[:2]
        # 使用 字典的.get() 方法，如果在映射中找不到前缀，返回 '其他类别'
        return code_to_group_map.get(code, '其他类别')

    print(f"--- 1.2 分析{df.name}学生每日食谱五大类别食物是否齐全 ---")
    # 应用函数，创建 '食物类别' 新列
    food_category_series = df['食物编码'].apply(lambda x: get_major_food_group_from_code(x, code_to_group_map))
    df_temp = food_category_series.to_frame(name='食物类别')
    # 获取食谱中实际包含的五大类别（排除“其他类别”）

    categories_present = df_temp[df_temp['食物类别'] != '其他类别']['食物类别'].unique().tolist()

    # 检查哪些五大类别是包含的，哪些是不包含的
    print("五大类别食物包含情况:")
    missing_categories = []
    for group_name in five_major_food_names:
        is_present = group_name in categories_present
        print(f"   - {group_name}: {'包含 ✅' if is_present else '不包含 ❌'}")
        if not is_present:
            missing_categories.append(group_name)

    # 判断是否五大类别齐全
    all_five_present = len(missing_categories) == 0
    print(f"五大类别是否齐全: {'是 ✅' if all_five_present else '否 ❌'}")

    # 输出缺少的类别（如果存在）
    if missing_categories:
        print(f"缺少的五大类别: {', '.join(missing_categories)}")
    else:
        print("所有五大类别都已包含。")

    # 检查是否有食物被归类到“其他类别”
    other_category_items = df[df_temp['食物类别'] == '其他类别']['主要成分'].unique().tolist()
    if other_category_items:
        print("-" * 50)
        print(f"以下主要成分未能归入五大类别，被标记为 '其他类别':\n {', '.join(other_category_items)}")


get_food_categories(df_woman)

--- 1.2 分析女学生每日食谱五大类别食物是否齐全 ---
五大类别食物包含情况:
   - 谷、薯类: 包含 ✅
   - 蔬菜、菌藻、水果类: 包含 ✅
   - 畜、禽、鱼、蛋类及制品: 包含 ✅
   - 奶、干豆、坚果、种子类及制品: 包含 ✅
   - 植物油类: 不包含 ❌
五大类别是否齐全: 否 ❌
缺少的五大类别: 植物油类
--------------------------------------------------
以下主要成分未能归入五大类别，被标记为 '其他类别':
 豆油


In [43]:
get_food_categories(df_man)

--- 1.2 分析男学生每日食谱五大类别食物是否齐全 ---
五大类别食物包含情况:
   - 谷、薯类: 包含 ✅
   - 蔬菜、菌藻、水果类: 包含 ✅
   - 畜、禽、鱼、蛋类及制品: 包含 ✅
   - 奶、干豆、坚果、种子类及制品: 包含 ✅
   - 植物油类: 不包含 ❌
五大类别是否齐全: 否 ❌
缺少的五大类别: 植物油类
--------------------------------------------------
以下主要成分未能归入五大类别，被标记为 '其他类别':
 豆油, 芝麻油


### 1.3 食物种类是否大于12种

In [44]:
def count_food_types(df):
    print(f"--- 1.3 {df.name}学生每日食谱食物种类数量分析 ---")
    unique_food_types_count = df['食物名称'].nunique()
    print(f"食物种类数量: {unique_food_types_count} 种")
    print(f"（要求日食谱 > 12 种）")
    if unique_food_types_count > 12:
        print("食物种类数量达标 ✅")
    else:
        print("每日食物种类数量不达标 ❌")

In [45]:
count_food_types(df_man)

--- 1.3 男学生每日食谱食物种类数量分析 ---
食物种类数量: 11 种
（要求日食谱 > 12 种）
每日食物种类数量不达标 ❌


## 2. 计算食谱的主要营养素含量

查出每100克可食部食物所含主要营养素的数量，从而算出食谱中各种主要营养素的含量

主要营养素: 碳水化合物、脂肪、蛋白质、矿物质、维生素、水

产能营养素: 碳水化合物、脂肪、蛋白质

## 3. 评价食谱提供的能量、餐次比及非产能主要营养素含量
根据
- 每日能量摄入目标
- 餐次比参考值
- 以及非产能主要营养素钙、铁、锌、维生素A、维生素B1、维生素B2、维生素C的参考摄入量

对食谱进行评价。


In [48]:
class Evaluation:
    def __init__(self, df, display_flag=False, rounding_decimals=2):
        self.df = df
        self.display_flag = display_flag
        self.rounding_decimals = rounding_decimals
        self.standards = {
            # 一日食谱种类最低数
            'daily_variety_min': 12,
            # 每日能量摄入目标及餐次比
            'energy_target': {'男': 2400, '女': 1900},  # kcal/d
            # 餐次比
            'meal_ratio_range': {'早餐': (0.25, 0.35), '午餐': (0.30, 0.40), '晚餐': (0.30, 0.40)},  # 注意这里使用附件4评价原则中的范围
            # 非产能营养素参考摄入量
            'micro_target': {  #
                '男': {'钙': 800, '铁': 12, '锌': 12.5, '维生素A': 800, '维生素B1': 1.4, '维生素B2': 1.4,
                       '维生素C': 100},
                # mg/d 或 μgRE/d
                '女': {'钙': 800, '铁': 20, '锌': 7.5, '维生素A': 700, '维生素B1': 1.2, '维生素B2': 1.2,
                       '维生素C': 100}
            },
            # 宏量营养素供能占比
            'macro_ratio_range': {'蛋白质': (0.10, 0.15), '脂肪': (0.20, 0.30), '碳水化合物': (0.50, 0.65)},
            'energy_conversion': {'蛋白质': 4, '脂肪': 9, '碳水化合物': 4},
            'aas_ref_pattern': {
                '异亮氨酸': 40,
                '亮氨酸': 70,
                '赖氨酸': 55,
                '含硫氨基酸': 35,  # 这是蛋氨酸和半胱氨酸的总参考值
                '芳香族氨基酸': 60,  # 这是苯丙氨酸和酪氨酸的总参考值
                '苏氨酸': 40,
                '色氨酸': 10,
                '缬氨酸': 50, },
            # ass评分
            'aas_eval_criteria': {
                '不合理': (0, 60),
                '不够合理': (60, 80),
                '比较合理': (80, 90),
                '合理': (90, float('inf')),  # 使用 float('inf') 表示无穷大
            },
            # 五大种类
            'five_major_food_names': ["谷、薯类",
                                      "蔬菜、菌藻、水果类",
                                      "畜、禽、鱼、蛋类及制品",
                                      "奶、干豆、坚果、种子类及制品",
                                      "植物油类"],
            # 五大种类的编码
            'five_major_food_groups': [
                {"谷类及制品": "01", "薯类、淀粉及制品": "02"},
                {"蔬菜类及制品": "04", "菌藻类": "05", "水果类及制品": "06"},
                {"畜肉类及制品": "08", "禽肉类及制品": "09", "鱼虾蟹贝类": "12", "蛋类及制品": "11"},
                {"乳类及制品": "10", "干豆类及制品": "03", "坚果、种子类": "07"},
                {"植物油类": "18"}
            ]
        }

        self.evaluation_results = {}
        # 建立编号和组名的映射
        self.code_to_group_map = {}
        # 使用正确的字符串键名访问 standards
        food_groups_data = self.standards.get("five_major_food_groups", [])  # 使用.get()并提供默认值，防止键不存在

        food_names_data = self.standards.get("five_major_food_names", [])
        self.code_to_group_map = {}

        for i, group_codes_dict in enumerate(food_groups_data):
            group_name = food_names_data[i]
            for _, code in group_codes_dict.items():
                self.code_to_group_map[code] = group_name

        self.standards["code_to_group_map"] = self.code_to_group_map

    #  --- 1 ---
    def process_and_evaluate_diet(self):

        def get_major_food_group_from_code(food_code, code_to_group_map):
            """
            根据食物编码的前两位查找对应的五大类别名称。
            非五大类别，则返回 '其他类别'。
            """
            code = food_code[:2]
            # 使用 字典的.get() 方法，如果在映射中找不到前缀，返回 '其他类别'
            return code_to_group_map.get(code, '其他类别')

        # 初始化 evaluation_results 字典
        # food_structure = {
        #     'categories_present': [],          # 列表：食谱中实际包含的五大类别的名称（不含“其他类别”）
        #     'missing_categories': [],          # 列表：标准五大类别中，食谱中缺少的类别名称
        #     'all_five_present': False,         # 布尔值：是否五大类别齐全
        #     'unique_food_types_count': 0,      # 整数：食谱中不重复的食物种类数量
        #     'daily_variety_met': False,        # 布尔值：食物种类数量是否达到每日推荐的最小种类数
        #     'other_category_items': [],        # 列表：被归类为“其他类别”的食物名称或主要成分列表
        #     # 'food_quantities_summary': None,   # 可选：食物名称及其总重量的摘要（例如 DataFrame 或 列表）
        # }

        # 复制 DataFrame 以便操作
        df_processed = self.df.copy()
        gender = self.df.name
        standards = self.standards
        # 分析食物结构 ---

        # 添加食物类别列
        # 确保 '食物编码' 列存在

        df_processed['食物类别'] = df_processed['食物编码'].apply(
            lambda x: get_major_food_group_from_code(x, standards['code_to_group_map'])
        )
        # 存在的五大种类

        categories_present = df_processed[df_processed['食物类别'] != '其他类别']['食物类别'].unique().tolist()
        # 缺少的五大种类
        missing_categories = [cat for cat in standards['five_major_food_names'] if cat not in categories_present]
        # 是否全部存在
        all_five_present = len(missing_categories) == 0

        # 统计食物种类数
        unique_food_types_count = df_processed['食物名称'].nunique()
        daily_variety_met = unique_food_types_count > standards['daily_variety_min']

        # 识别“其他类别”项目

        # “其他类别”项目的主要成分
        other_category_items = df_processed[df_processed['食物类别'] == '其他类别']['主要成分'].unique().tolist()

        # --- 将步骤 1 的分析结果存储到 evaluation_results['food_structure'] 字典中 ---
        self.evaluation_results['food_structure'] = {
            'categories_present': categories_present,
            'missing_categories': missing_categories,
            'all_five_present': all_five_present,
            'unique_food_types_count': unique_food_types_count,
            'daily_variety_met': daily_variety_met,
            'other_category_items': other_category_items,
        }

    #  --- 2 计算主要营养素含量 ---
    def calculate_nutrient_intakes(self):
        """
        :return: df_intake:DataFrame（一日各营养素摄入量及能量总量）
        :return: df_meal:DataFrame（餐次的各营养素摄入量及能量总量）
        """
        df = self.df
        weight_col = '食物重量(克)'
        if not weight_col in df.columns:
            df[weight_col] = df['可食部（克/份）'] * df['食用份数']

        # --- 2.1 计算食谱的主要营养素摄入量 ---

        # 定义需要计算摄入量的营养素列名列表
        nutrient_cols_to_calculate = df_man_origin.columns.to_list()[6:]

        # 计入后的摄入量列添加到df中
        for nutrient_col_100g in nutrient_cols_to_calculate:
            # 提取营养素名称和单位 (例如 '蛋白质', 'g')
            parts = nutrient_col_100g.replace(')', '').split('(')
            # 营养素名字
            nutrient_name = parts[0].strip()
            # 单位名称
            unit_info = parts[1].strip()  # 例如 'g/100g', 'mg/100g'
            # 摄入单位
            intake_unit = unit_info.split('/')[0]  # 例如 'g', 'mg'
            # 构建新的摄入量列名
            intake_col_name = f'{nutrient_name}摄入量 ({intake_unit})'

            # 使用向量化计算： (总克重 / 100) * 每100克含量
            # 创建新列 列名: 营养素名称
            df[intake_col_name] = (df[weight_col] / 100) * df[nutrient_col_100g]

        # --- 2.2 计算一日总营养素摄入量 ---

        # 添加所有新计算的列的列名称为列表
        intake_cols = [col for col in df.columns if '摄入量 (' in col and col.endswith(')')]

        # 营养素每天的摄入量
        df_intake = df[intake_cols].agg(['sum'])

        # --- 2.3 计算能量 ---

        energy_conversion = {'蛋白质': 4, '脂肪': 9, '碳水化合物': 4}  # kcal/g

        substance_col = [col for col in df.columns if col.find("摄入量") > 0]

        energy_substance = [col + '能量摄入量 (kcal)' for col in energy_conversion.keys()]

        # energy_conversion = {'蛋白质': 4, '脂肪': 9, '碳水化合物': 4, '膳食纤维': 2}
        # 宏量营养素的能量
        for substance in energy_conversion.keys():
            df[substance + '能量摄入量 (kcal)'] = df[substance + '摄入量 (g)'] * energy_conversion[substance]

        # 将总能量添加到日摄入量字典中
        df_intake['总能量摄入量 (kcal)'] = df[energy_substance].sum().sum()

        # --- 2.4 计算每餐总营养素摄入量及能量 ---

        df_meal = df.groupby('餐次')[substance_col].sum()

        df_meal['总能量摄入量 (kcal)'] = df_meal[energy_substance].sum(axis=1)
        if self.display_flag:

            print(f"--- 2.1 完成计算{df.name}学生食谱的主要营养素含量 ---")

            print(f"--- 2.2 完成计算{df.name}学生一日营养素总摄入量 ---")
            for col in df_intake.iloc[:-1].columns:
                parts = col.replace(')', '').split('(')
                name = parts[0].strip()
                # 单位名称
                unit = parts[1].strip()  # 例如 'g/100g', 'mg/100g'
                print(f"    {name}为:{df_intake[col].values[0]} {unit} ")

            print(f"--- 2.4 完成计算{df.name}学生一日能量总摄入量 ---")
            print(f"    一日总能量摄入量: {df_intake['总能量摄入量 (kcal)'].values[0]:.2f} kcal")

            print(f"--- 2.4 完成计算{df.name}学生一日每餐次总能量摄入量 ---")
            print(df_meal[['总能量摄入量 (kcal)']].round(2))

        return df_intake, df_meal

    #  --- 3 评价 ---
    # --- 3.1 评价函数：评价能量 ---
    def evaluate_energy(self, df_intake: pd.DataFrame) -> None:
        """
        评价一日总能量摄入量是否符合个体需要 (根据附件4评价原则1)。
        Returns:
            energy_evaluation：存储实际每日能量摄入量, 每日目标摄入量, 相差百分比, 评价。
        """
        gender = self.df.name

        rounding_decimals = self.rounding_decimals
        # 分析结果
        energy_evaluation = {}

        # 获取实际每日总能量摄入量
        actual_energy_kcal = df_intake['总能量摄入量 (kcal)'].values[0]

        # 获取目标性别
        target_energy_kcal = self.standards['energy_target'].get(gender)

        # 存储实际摄入量，并进行四舍五入
        energy_evaluation['total_energy_kcal'] = round(actual_energy_kcal, rounding_decimals)
        # 存储目标摄入量
        energy_evaluation['target_kcal'] = target_energy_kcal
        # 打印评价头部信息
        print("--- 3.1 能量评价 ---")
        print(f"    实际值: {actual_energy_kcal:.{rounding_decimals}f} kcal")
        print(f"    目标值: {target_energy_kcal:.{rounding_decimals}f} kcal")

        # 计算(实际摄入 - 目标摄入) 占 目标摄入 的百分比的绝对值
        percentage = ((actual_energy_kcal - target_energy_kcal) / target_energy_kcal) * 100
        print(f"    实际值与目标值的差值占目标值的百分比: {percentage:.0f}%")

        # 存储并四舍五入百分比
        energy_evaluation['percentage'] = round(percentage, rounding_decimals)
        # --- 评价结论 ---
        # 百分比的绝对值不超过10%即合格
        if abs(percentage) < 10:
            energy_evaluation['comment'] = "达标"
            print("    评价: 每日能量摄入目标在目标范围内 ✅")
        else:  #  < 0.1
            energy_evaluation['comment'] = "不达标"
            print("    评价: 每日能量摄入目标不在目标范围内 ❌")
        self.evaluation_results['energy_evaluation'] = energy_evaluation

    # --- 3.2 评价函数：评价餐次比 ---
    def evaluate_meal_ratio(self, df_intake: pd.DataFrame, df_meal: pd.DataFrame) -> None:
        """
        评价三餐供能比是否在推荐范围内 (根据附件4评价原则2)。
        Args:
            df_meal: 包含每餐总营养素摄入量的 DataFrame（包含 '总能量摄入量 (kcal)' 列）。
            df_meal: 包含一日总能量的 DataFrame（包含 '总能量摄入量 (kcal)' 列）。

        Returns:
            meal_ratio_evaluation: 包含每餐供能比评价结果的字典。'meal_ratios' 键下是各餐次的评价结果，还有 'total_daily_energy' 和 'overall_comment'。
        """
        rounding_decimals = self.rounding_decimals
        meal_ratio_evaluation = {}

        # 获取一日总能量，如果不存在或为0则无法计算餐次比
        meal_energy_col = '总能量摄入量 (kcal)'
        total_daily_energy = df_intake[meal_energy_col].values[0]

        # 获取餐次比目标范围字典
        meal_ratio_ranges = self.standards.get('meal_ratio_range')

        meal_ratio_evaluation['total_daily_energy'] = round(total_daily_energy, rounding_decimals)

        meal_ratio_evaluation['meal_ratios'] = {}

        meal_ratio_evaluation['overall_comment'] = "达标 ✅"  # 乐观初始化整体评价

        print("\n--- 3.2 餐次比评价 ---")
        # 定义常见的餐次顺序，以便按顺序打印和评价
        meal_order = ['早餐', '午餐', '晚餐']

        # --- 遍历每餐进行评价 ---
        for meal in meal_order:
            # 从 meal_intake DataFrame 中获取该餐的能量，如果该餐不存在则为 0
            meal_energy = df_meal.loc[meal, meal_energy_col]
            # 计算该餐能量占日总能量的百分比 (转换为 0-1 范围的小数)
            meal_percentage = (meal_energy / total_daily_energy)
            # 获取该餐的推荐范围 (0-1 范围的小数对)
            target_range = meal_ratio_ranges.get(meal)

            meal_eval = {
                'actual_kcal': round(meal_energy, rounding_decimals),  # 存储实际能量
                'actual_percentage': round(meal_percentage * 100, rounding_decimals),  # 存储实际百分比 (0-100)
                'target_range_percent': None,  # 存储目标范围百分比 (0-100)
                'comment': "标准缺失"  # 初始化评价结论
            }
            # 餐次: meal_eval, meal_eval是可变对象, 修改会继续保留
            meal_ratio_evaluation['meal_ratios'][meal] = meal_eval  # 将单餐评价结果添加到字典中

            # 计算目标范围
            target_min_percent = target_range[0] * 100  # 转换为 0-100 范围的百分比
            target_max_percent = target_range[1] * 100  # 转换为 0-100 范围的百分比

            meal_eval['target_range_percent'] = (
                round(target_min_percent, 2), round(target_max_percent, 2))  # 存储目标范围 (保留2位)
            print(f"  {meal} 供能:")
            print(
                #   实际功能: xxx kcal (百分比)
                f"    实际供能: {meal_energy:.{rounding_decimals}f} kcal (实际功能占比: {meal_eval['actual_percentage']:.{rounding_decimals}f}%)")  # 打印实际百分比，保留2位

            print(f"    目标供能占比范围: {target_min_percent:.0f}% - {target_max_percent:.0f}%")  # 打印目标范围，不带小数

            # --- 评价结论 (判断是否在目标范围内) ---
            if target_range[0] <= meal_percentage <= target_range[1]:
                meal_eval['comment'] = "达标"
                print(f"    评价: 达标 ✅")

            else:
                meal_eval['comment'] = "不达标"
                print("    评价: 不达标 ❌")
                # 如果有任何一餐偏离，整体评价就不是“达标”了
                if meal_ratio_evaluation['overall_comment'] == "达标 ✅":
                    meal_ratio_evaluation['overall_comment'] = "部分餐次比偏离"
        self.evaluation_results['meal_ratio_evaluation'] = meal_ratio_evaluation

    # --- 3.3 评价函数：评价非产能主要营养素 ---
    def evaluate_micronutrients(self, df_intake: pd.DataFrame) -> None:
        """
        评价非产能主要营养素摄入量是否达到标准 (根据附件4评价原则3)。
        Args:
            df-intake: 包含一日总营养素摄入量的字典。营养素列格式为 '营养素名称摄入量 (单位)' 格式。

        Returns:
            包含非产能营养素评价结果的字典。'micronutrients' 键下是每个营养素的评价详情，还有 'overall_comment'。
        """
        gender = self.df.name
        rounding_decimals = self.rounding_decimals
        micronutrients_evaluation = {}

        # 获取该性别的非产能营养素目标字典
        micronutrient_targets = self.standards['micro_target'].get(gender)

        print("\n--- 3.3 非产能主要营养素评价 ---")

        micronutrients_evaluation['micronutrients'] = {}
        micronutrients_evaluation['overall_comment'] = "基本达标 😊"  # 乐观初始化整体评价

        # 需要评价的非产能营养素名称列表，从该性别的标准中获取
        micros_to_evaluate = list(micronutrient_targets.keys())

        # --- 遍历每个非产能营养素进行评价 ---
        for micro_name in micros_to_evaluate:
            target_amount = micronutrient_targets.get(micro_name)  # 获取该营养素的目标量

            actual_intake_col_name = [col for col in df_intake.columns if col.startswith(micro_name)][
                0]  # 实际摄入量的键名 (如 '钙摄入量 (mg)')
            actual_intake = df_intake[actual_intake_col_name].values[0]  # 实际摄入量的值

            micro_eval = {
                'actual_intake': None,  # 实际摄入量
                'target_amount': target_amount,  # 目标摄入量
                'unit': None,  # 实际摄入量单位
                'percentage_of_rni_ai': None,
                'comment': "数据缺失或标准缺失"  # 初始化评价结论
            }

            micronutrients_evaluation['micronutrients'][micro_name] = micro_eval  # 将单营养素评价结果添加到字典中

            print(f"  {micro_name} 摄入量:")

            # --- 评价结论  ---

            # 存储 实际谁让量
            micro_eval['actual_intake'] = round(actual_intake, rounding_decimals)
            # 提取实际摄入量单位
            actual_unit_match = actual_intake_col_name.split('(')[-1].replace(')', '').strip()
            micro_eval['unit'] = actual_unit_match

            print(f"    实际摄入量: {actual_intake:.{rounding_decimals}f} {actual_unit_match}")
            print(f"    目标量: {target_amount:.2f} {actual_unit_match}")  # 打印目标，假设单位一致

            if target_amount > 0:
                # 计算占目标比例
                percentage = (actual_intake / target_amount) * 100
                micro_eval['percentage_of_rni_ai'] = round(percentage, rounding_decimals)
                print(f"    实际值占目标值的比例: {percentage:.2f}%")

                # 评价结论 (达到或超过 RNI/AI 为达标，否则不足)
                if percentage >= 100:
                    micro_eval['comment'] = "达标或偏高"
                    print("    评价: 达标或偏高 ✅")

                else:  # percentage < 100
                    micro_eval['comment'] = "不足"
                    print("    评价: 不足 ❌")
                    # 如果有任何一个不足，整体评价就不是“基本达标”了
                    if micronutrients_evaluation['overall_comment'] == "基本达标 😊":
                        micronutrients_evaluation['overall_comment'] = "部分非产能营养素摄入不足"
        self.evaluation_results['micronutrients_evaluation'] = micronutrients_evaluation

    #  --- 4 评价 ---
    # 评价函数：评价一日食谱的宏量营养素供能比是否在推荐范围内 (根据附件4评价原则4)。
    def evaluate_macro_ratios(self, df_intake: pd.DataFrame) -> None:
        """
        评价一日食谱的宏量营养素供能比是否在推荐范围内 (根据附件4评价原则4)。
        Args:
            df_intake: 包含一日总营养素摄入量和能量的Dataframe。需包含 '总能量摄入量 (kcal)' 和 '蛋白质摄入量 (g)'、'脂肪摄入量 (g)'、'碳水化合物摄入量 (g)' 键。
        Returns:
            包含宏量营养素供能比评价结果的字典。'macro_ratios' 键下是各宏量的评价详情，还有 'overall_comment'。
        """

        rounding_decimals = self.rounding_decimals
        macro_ratios_evaluation = {}
        # 从 daily_intake 字典中获取日总能量和宏量摄入量 (克)
        macros_to_evaluate = ['蛋白质', '脂肪', '碳水化合物']

        total_daily_energy = df_intake['总能量摄入量 (kcal)'].iloc[0]

        macro_ratio_ranges = self.standards.get('macro_ratio_range')
        energy_conversion = self.standards.get('energy_conversion')

        print("\n--- 4 宏量营养素供能比评价 ---")

        macro_ratios_evaluation['macro_ratios'] = {}
        macro_ratios_evaluation['overall_comment'] = "达标 ✅"  # 乐观初始化整体评价

        # 计算蛋白质、脂肪、碳水化合物各自贡献的能量 (kcal)
        macro_energy_dict = {}
        for macro_name in macros_to_evaluate:
            actual_col_name = [col for col in df_intake.columns if col.startswith(macro_name)][0]
            macro_energy_dict[macro_name] = df_intake[actual_col_name].iloc[0] * energy_conversion.get(macro_name, 0)

        # 计算各项宏量占“总能量”的比例 (0-1 范围)

        # --- 遍历宏量营养素进行评价 ---
        for macro_name in macros_to_evaluate:

            actual_kcal = macro_energy_dict.get(macro_name, 0)

            # 计算实际供能比 (0-1 范围)
            actual_ratio = actual_kcal / total_daily_energy
            # 获取该宏量的推荐范围 (0-1 范围)
            target_ratio = macro_ratio_ranges.get(macro_name)

            macro_eval = {
                'actual_kcal': round(actual_kcal, rounding_decimals),  # 实际贡献能量
                'actual_ratio': round(actual_ratio, rounding_decimals),  # 实际比例 (0-1)
                'actual_percentage': round(actual_ratio * 100, rounding_decimals),  # 实际百分比 (0-100)
                'target_ratio_range': target_ratio,  # 目标范围 (0-1)
                'target_ratio_range_percent': None,  # 目标范围百分比 (0-100)
                'comment': "标准缺失"  # 初始化评价结论
            }
            macro_ratios_evaluation['macro_ratios'][macro_name] = macro_eval

            print(f"  {macro_name} 供能比:")

            target_min_percent = target_ratio[0] * 100  # 转换为 0-100 百分比
            target_max_percent = target_ratio[1] * 100  # 转换为 0-100 百分比

            macro_eval['target_ratio_range_percent'] = (
                round(target_min_percent, 2), round(target_max_percent, 2))  # 存储目标范围百分比 (保留2位)

            print(f"    实际: {macro_eval['actual_percentage']:.{rounding_decimals}f}%")  # 打印实际百分比，保留2位
            print(f"    目标范围: {target_min_percent:.0f}% - {target_max_percent:.0f}%")  # 打印目标范围，不带小数

            # --- 评价结论 (判断是否在目标范围内) ---
            if target_ratio[0] <= actual_ratio <= target_ratio[1]:
                macro_eval['comment'] = "达标"
                print("    评价: 达标 ✅")
            elif actual_ratio < target_ratio[0]:
                macro_eval['comment'] = "偏低"
                print("    评价: 偏低 ❌")
                # 如果有任何一个偏离，整体评价就不是“达标”了
                if macro_ratios_evaluation['overall_comment'] == "达标 ✅":
                    macro_ratios_evaluation['overall_comment'] = "部分宏量比偏离"
            else:  # actual_ratio > target_range[1]
                macro_eval['comment'] = "偏高"
                print("    评价: 偏高 ❌")
                if macro_ratios_evaluation['overall_comment'] == "达标 ✅":
                    macro_ratios_evaluation['overall_comment'] = "部分宏量比偏离"
        # # --- 可选：校验宏量能量总和与日总能量是否一致 ---
        # # P+F+C+Fiber 的能量总和理论上应该等于总能量
        # 这里使用 round() 避免浮点误差导致校验失败
        sum_all_macro_fiber_kcal = macro_energy_dict['蛋白质'] + macro_energy_dict['脂肪'] + macro_energy_dict[
            '碳水化合物']
        if total_daily_energy > 0 and abs(
                sum_all_macro_fiber_kcal - total_daily_energy) / total_daily_energy > 0.01:  # 允许1%的误差
            print("警告: 计算的宏量营养素能量总和与日总能量不符，请检查计算或数据。")
        self.evaluation_results['macro_ratios_evaluation'] = macro_ratios_evaluation

    # ---  5 评价每餐的蛋白质氨基酸评分 (AAS) ---

    # 计算 AAS 需要每餐的蛋白质总量（克）和每种必需氨基酸总量（毫克），以及 AAS 参考模式（毫克/克蛋白质）。

    def calculate_and_evaluate_per_meal_aas(self, df_meal: pd.DataFrame) -> None:
        """
        计算并评价每餐的蛋白质氨基酸评分 (AAS) (根据附件4评价原则5)。
        Args:
            df_meal: 包含每餐总营养素摄入量（包括蛋白质和必需氨基酸）的 DataFrame。
                         应包含 '蛋白质摄入量 (g)' 列和必需氨基酸摄入量列。
                         必需氨基酸列名例如 '异亮氨酸摄入量 (g)' 或 '异亮氨酸摄入量 (mg)'。


        Returns:
            包含每餐 AAS 计算和评价结果的字典。'meal_aas' 键下是各餐次的详情，还有 'overall_comment'。
        """
        rounding_decimals = self.rounding_decimals
        ass_evaluation = {}
        aas_ref_pattern = self.standards.get('aas_ref_pattern')
        aas_eval_criteria = self.standards.get('aas_eval_criteria')

        print("\n--- 5 每餐蛋白质氨基酸评分 (AAS) 评价 ---")

        # 识别计算 AAS 需要的列：蛋白质（g）和必需氨基酸摄入量

        protein_intake_col_g = '蛋白质摄入量 (g)'
        essential_aa_names = list(aas_ref_pattern.keys())  # 从参考模式中获取必需氨基酸名称

        # 构建必需氨基酸的可能摄入量列名
        essential_aa_intake_cols = [f"{aa_name}摄入量 (g)" for aa_name in essential_aa_names]

        ass_evaluation['meal_aas'] = {}
        ass_evaluation['overall_comment'] = "达标 ✅"  # 乐观初始化 (所有餐次都 '合理' 或 '比较合理')

        # --- 计算和评价每餐 AAS ---
        meal_order = ['早餐', '午餐', '晚餐']  # 按顺序处理

        overall_aas_sufficient = True  # 跟踪是否所有餐次 AAS 都达到了 '比较合理' 或 '合理'

        for meal in meal_order:

            meal_protein_g = df_meal.loc[meal, protein_intake_col_g]  # 该餐次的蛋白质摄入量 (克)

            print(f"  {meal} (蛋白质: {meal_protein_g:.{rounding_decimals}f}g):")

            limiting_aa = None  # 限制性氨基酸名称
            min_ratio = float('inf')  # 最小的氨基酸比例
            aas_score = None  # 计算得到的 AAS

            aa_ratios = {}  # 存储该餐次各必需氨基酸的比例 (实际/目标)

            # 计算该餐次各必需氨基酸的比例
            for aa_intake_col in essential_aa_intake_cols:
                aa_name = aa_intake_col.split('摄入量')[0].strip()  # 提取氨基酸名称 (例如 '异亮氨酸')
                aa_unit = aa_intake_col.split('(')[-1].replace(')', '').strip()  # 提取单位 (例如 'g' 或 'mg')

                aa_intake_amount = df_meal.loc[meal, aa_intake_col]  # 该餐次的氨基酸摄入量 (可能是 g 或 mg)

                # 将氨基酸摄入量转换为毫克 (mg)
                if aa_unit.lower() == 'g':
                    aa_intake_mg = aa_intake_amount * 1000
                elif aa_unit.lower() == 'mg':
                    aa_intake_mg = aa_intake_amount
                else:
                    # 如果单位不是 g 也不是 mg，打印警告并跳过该氨基酸
                    print(f"    警告：氨基酸 '{aa_name}' 的单位 '{aa_unit}' 不支持，跳过其比例计算。")
                    continue  # 跳过该氨基酸

                # 获取该氨基酸在参考模式中的值 (mg/g 蛋白质)
                ref_amount_mg_per_g_protein = aas_ref_pattern.get(aa_name, None)

                if ref_amount_mg_per_g_protein is None:
                    # 如果该氨基酸没有参考模式值
                    print(f"    警告：氨基酸 '{aa_name}' 没有参考模式值，跳过其比例计算。")
                    continue  # 跳过该氨基酸
                if ref_amount_mg_per_g_protein <= 0:
                    # 如果参考模式值为零或负数，也跳过
                    print(f"    警告：氨基酸 '{aa_name}' 的参考模式值为零或负数，跳过其比例计算。")
                    continue

                # --- 计算该氨基酸的比例 ---
                # 比例 = (氨基酸摄入量 mg) / (蛋白质摄入量 g * 参考模式 mg/g)
                ratio = aa_intake_mg / (meal_protein_g * ref_amount_mg_per_g_protein)
                aa_ratios[aa_name] = round(ratio, rounding_decimals)
                print(
                    f"      {aa_name} 比例: {ratio:.2f} (实际 {aa_intake_mg:.2f}mg / 目标 {meal_protein_g * ref_amount_mg_per_g_protein:.2f}mg)")

                # 更新最小比例和限制性氨基酸
                if ratio < min_ratio:
                    min_ratio = ratio
                    limiting_aa = aa_name

            # --- 计算 AAS Score ---
            if min_ratio != float('inf'):  # 确保至少有一个氨基酸参与了计算
                aas_score = round(min_ratio * 100, rounding_decimals)  # AAS = 最小比例 * 100

            ass_evaluation['meal_aas'][meal] = {
                'protein_g': round(meal_protein_g, rounding_decimals),
                'aas': aas_score,
                'limiting_aa': limiting_aa,
                'comment': "计算失败或标准缺失",  # 初始化评价结论
                'aa_ratios': aa_ratios  # 存储各氨基酸比例
            }

            # --- 评价 AAS 等级 ---
            print(f"    AAS: {aas_score if aas_score is not None else 'N/A':.{rounding_decimals}f}")
            if aas_score is not None:
                print(f"    限制性氨基酸: {limiting_aa}")

                meal_aas_comment = "未知等级"  # 默认等级
                is_current_meal_sufficient = False  # 跟踪当前餐次是否达到了 '比较合理' 或 '合理'

                # 遍历评价标准等级范围，找到 AAS 所属的等级
                # criteria 示例: '不合理': (0, 60), '不够合理': (60, 80), '比较合理': (80, 90), '合理': (90, float('inf'))
                # 需要确保标准中的范围是排好序且连续的，以便正确匹配
                # 最佳实践是对 criteria 的键值对按范围下限进行排序

                sorted_criteria = sorted(aas_eval_criteria.items(), key=lambda item: item[1][0])

                for comment, range_tuple in sorted_criteria:
                    if range_tuple[0] <= aas_score < range_tuple[1]:
                        meal_aas_comment = comment
                        # 判断是否达到“比较合理”或“合理”
                        if comment in ['合理', '比较合理']:
                            is_current_meal_sufficient = True
                        break  # 找到匹配等级，跳出循环

                ass_evaluation['meal_aas'][meal]['comment'] = meal_aas_comment
                print(f"    评价: {meal_aas_comment}")

                if not is_current_meal_sufficient:  # 如果当前餐次不是 '合理' 或 '比较合理'
                    overall_aas_sufficient = False  # 整体 AAS 就不是“达标”了

            else:
                # AAS 计算失败的情况
                print("    评价: AAS 计算失败。")
                overall_aas_sufficient = False  # 计算失败视为未达标

        # --- 整体 AAS 评价 ---
        if overall_aas_sufficient:
            ass_evaluation['overall_comment'] = "所有餐次 AAS 达标 ✅"  # 如果所有处理的餐次都 '比较合理' 或 '合理'
        else:
            ass_evaluation['overall_comment'] = "部分餐次 AAS 不足、计算失败或数据缺失 ❌"  # 如果有任何餐次未达标或数据有问题
        self.evaluation_results['ass_evaluation'] = ass_evaluation

    # --- 6 综合所有评价结果，生成整体评价和膳食建议 ---
    def generate_overall_evaluation_and_suggestions(self):
        """
        综合所有评价结果，生成整体评价和膳食建议。

        Args:
            evaluation_results: 包含所有评价步骤结果的字典。
                                应包含
                                'food_structure',  1 食物结构结果
                                'energy_evaluation', 3.1 能量评价结果
                                'meal_ratio_evaluation', 3.2 餐次比评价结果
                                'micronutrients_evaluation', 3.3 非产能主要营养素评价结果
                                'macro_ratios_evaluation', 4 宏量营养素供能比
                                "ass_evaluation",5 蛋白质氨基酸 评分
        Returns:
            格式化的整体评价和建议字符串。
        """
        df = self.df
        gender = df.name
        standards = self.standards
        evaluation_results = self.evaluation_results

        rounding_decimals = self.rounding_decimals

        overall_summary = f"\n--- {gender} 食谱整体评价与建议 ---\n"

        # --- 总结各项评价 ---

        overall_summary += "\n各项评价总结:\n"

        # 1. 食物结构总结
        fs_eval = evaluation_results.get('food_structure', {})
        if fs_eval:
            overall_summary += f"  食物种类: {fs_eval.get('unique_food_types_count', 'N/A')} 种 (目标 > {standards.get('daily_variety_min', 'N/A')} 种) - 评价: {'达标 ✅' if fs_eval.get('daily_variety_met', False) else '不足 ❌'}\n"
            overall_summary += f"  五大类包含: {'齐全 ✅' if fs_eval.get('all_five_present', False) else '不齐全 ❌'}\n"
            if fs_eval.get('missing_categories'):
                overall_summary += f"    缺少的类别: {', '.join(fs_eval['missing_categories'])}\n"
            if fs_eval.get('other_category_items'):
                overall_summary += f"    未归类食物: {', '.join(fs_eval['other_category_items'])}\n"
        else:
            overall_summary += "  食物结构评价数据缺失。\n"

        # 3.1 能量摄入总结
        energy_eval = evaluation_results.get('energy_evaluation', {})
        if energy_eval:
            # 使用正确的键 'total_energy_kcal' 和 'percentage'
            actual_energy_kcal = energy_eval.get('total_energy_kcal', 'N/A')
            target_energy_kcal = energy_eval.get('target_kcal')
            percentage = energy_eval.get('percentage', 'N/A')  # 获取存储的百分比

            summary_line = f"  总能量: {actual_energy_kcal:.{rounding_decimals}f} kcal"
            if target_energy_kcal is not None:
                # 打印占目标的百分比时，直接使用获取到的 percentage
                summary_line += f" (目标 {target_energy_kcal:.0f} kcal, 占目标 {percentage:.1f}%)"
            summary_line += f" - 评价: {energy_eval.get('comment', 'N/A')}\\n"
            overall_summary += summary_line

        else:
            overall_summary += "  能量评价数据缺失。\\n"

        # 3.2 餐次比总结
        meal_ratio_eval = evaluation_results.get('meal_ratio_evaluation', {})
        if meal_ratio_eval and meal_ratio_eval.get('meal_ratios'):
            overall_summary += "  餐次供能比:\n"
            for meal, eval_data in meal_ratio_eval['meal_ratios'].items():
                summary_line = f"    {meal}: {eval_data.get('actual_percentage', 'N/A'):.1f}%"
                if eval_data.get('target_range_percent'):
                    summary_line += f" (目标 {eval_data['target_range_percent'][0]:.0f}%-{eval_data['target_range_percent'][1]:.0f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  餐次比整体评价: {meal_ratio_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "  餐次比评价数据缺失。\n"

        # 3.3 非产能主要营养素总结
        micro_eval = evaluation_results.get('micronutrients_evaluation', {})
        if micro_eval and micro_eval.get('micronutrients'):
            overall_summary += "  非产能营养素:\n"
            for micro, eval_data in micro_eval['micronutrients'].items():
                summary_line = f"  {micro}: {eval_data.get('actual_intake', 'N/A'):.{rounding_decimals}f}{eval_data.get('unit', '?')}"
                if eval_data.get('target_amount') is not None:
                    summary_line += f" (目标 {eval_data['target_amount']:.0f}{eval_data.get('unit', '?')}, 占目标 {eval_data.get('percentage_of_rni_ai', 'N/A'):.1f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  非产能营养素整体评价: {micro_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "  非产能主要营养素评价数据缺失。\n"

        # 4. 宏量营养素供能比总结
        macro_eval = evaluation_results.get('macro_ratios_evaluation', {})
        if macro_eval and macro_eval.get('macro_ratios'):
            overall_summary += "  宏量供能比:\n"
            for macro, eval_data in macro_eval['macro_ratios'].items():
                summary_line = f"    {macro}: {eval_data.get('actual_percentage', 'N/A'):.1f}%"
                if eval_data.get('target_range_percent'):
                    summary_line += f" (目标 {eval_data['target_range_percent'][0]:.0f}%-{eval_data['target_range_percent'][1]:.0f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  宏量供能比整体评价: {macro_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "  宏量营养素供能比评价数据缺失。\n"

        # 5. AAS 总结
        aas_eval = evaluation_results.get('ass_evaluation', {})
        if aas_eval and aas_eval.get('meal_aas'):
            overall_summary += "  每餐 AAS:\n"
            for meal, eval_data in aas_eval['meal_aas'].items():
                summary_line = f"    {meal}: AAS {eval_data.get('aas', 'N/A'):.1f}"
                if eval_data.get('limiting_aa'):
                    summary_line += f" (限制性氨基酸: {eval_data['limiting_aa']})"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  每餐 AAS 整体评价: {aas_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "  每餐 AAS 评价数据缺失或计算失败。\n"

        # --- 综合评价结论和膳食建议 ---
        overall_summary += "\n综合评价结论:\n"
        suggestions = []  # Collect suggestions

        # --- 根据各项评价结果生成具体的建议 ---
        # 食物结构建议
        if fs_eval and not fs_eval.get('daily_variety_met', False):
            suggestions.append(
                f"食谱种类不足，建议增加每日食物种类数量，目标 > {standards.get('daily_variety_min', 'N/A')} 种，以增加食物多样性。")
        if fs_eval and not fs_eval.get('all_five_present', False):
            suggestions.append(
                f"食谱包含的食物类别不全，建议增加 {', '.join(fs_eval.get('missing_categories', []))} 等五大类食物的摄入。")

        # 能量建议
        energy_comment = energy_eval.get('comment')
        if energy_comment == '偏低':
            suggestions.append(
                f"总能量摄入偏低 ({energy_eval.get('actual_kcal', 'N/A'):.0f} kcal)，建议适量增加食物摄入总量。")
        elif energy_comment == '偏高':
            suggestions.append(
                f"总能量摄入偏高 ({energy_eval.get('actual_kcal', 'N/A'):.0f} kcal)，建议适量减少食物摄入总量。")

        # 餐次比建议
        if meal_ratio_eval and meal_ratio_eval.get('overall_comment') == '部分餐次比偏离':
            for meal, eval_data in meal_ratio_eval.get('meal_ratios', {}).items():
                meal_comment = eval_data.get('comment')
                if meal_comment == '偏低':
                    suggestions.append(
                        f"{meal} 供能比偏低 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议增加 {meal} 的食物摄入量，以使供能更均衡。")
                elif meal_comment == '偏高':
                    suggestions.append(
                        f"{meal} 供能比偏高 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议减少 {meal} 的食物摄入量，并合理分配到其他餐次。")

        # 宏量供能比建议
        if macro_eval and macro_eval.get('overall_comment') == '部分宏量比偏离':
            for macro, eval_data in macro_eval.get('macro_ratios', {}).items():
                macro_comment = eval_data.get('comment')
                if macro_comment == '偏低':
                    suggestions.append(
                        f"{macro} 供能比偏低 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议增加富含 {macro} 的食物摄入，如{'全谷物、薯类' if macro == '碳水化合物' else ('鱼禽蛋瘦肉、豆制品' if macro == '蛋白质' else '优质植物油、坚果')}等。")
                elif macro_comment == '偏高':
                    suggestions.append(
                        f"{macro} 供能比偏高 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议减少富含 {macro} 的食物摄入。")

        # 非产能营养素建议
        if micro_eval and micro_eval.get('overall_comment') == '部分非产能营养素摄入不足':
            suggestions.append("食谱中部分非产能主要营养素摄入不足，需重点改进：")
            for micro, eval_data in micro_eval.get('micronutrients', {}).items():
                micro_comment = eval_data.get('comment')
                if micro_comment == '不足' or micro_comment == '严重不足':
                    suggestions.append(
                        f"  {micro}: 摄入量不足 (占目标 {eval_data.get('percentage_of_rni_ai', 'N/A'):.1f}%)，建议增加富含 {micro} 的食物。")
                    # 这里可以根据营养素提供一些食物来源建议，但需要一个食物-营养素映射表，此处简化
                    if micro == '钙':
                        suggestions[-1] += " (如奶制品、豆制品、深绿色蔬菜)"
                    elif micro == '铁':
                        suggestions[-1] += " (如瘦肉、动物肝脏、木耳)"
                    elif micro == '锌':
                        suggestions[-1] += " (如贝壳类海产品、红色肉类)"
                    # ... 其他微量营养素建议 ...

        # AAS 建议
        if aas_eval and aas_eval.get('overall_comment') == '部分餐次 AAS 不足或数据缺失 ❌':
            suggestions.append("部分餐次蛋白质质量（AAS）不理想，需注意食物搭配：")
            for meal, eval_data in aas_eval.get('meal_aas', {}).items():
                aas_comment = eval_data.get('comment')
                if aas_comment in ['不合理', '不够合理']:
                    suggestions.append(
                        f"  {meal}: AAS 不理想 ({eval_data.get('aas', 'N/A'):.1f})，建议在该餐次搭配不同来源的蛋白质食物，如谷类和豆类同食，以提高蛋白质互补作用。")
                    if eval_data.get('limiting_aa'):
                        suggestions[-1] += f" 特别注意补充富含 {eval_data['limiting_aa']} 的食物。"

        # --- 生成最终报告 ---
        overall_summary += "\n膳食建议:\n"
        if suggestions:
            for i, suggestion in enumerate(suggestions):
                overall_summary += f"  {i + 1}. {suggestion}\n"
        else:
            overall_summary += "  食谱评价基本符合要求，请继续保持均衡膳食。\n"

        overall_summary += f"\n--- {gender} 食谱整体评价与建议结束 ---\n"

        # 直接打印整体评价和建议
        print(overall_summary)

        # return overall_summary  # 返回生成的总结字符串

    def main(self):
        self.process_and_evaluate_diet()
        # 2  计算主要营养素含量
        df_intake, df_meal = self.calculate_nutrient_intakes()
        # 3.1 评价函数：评价能量
        self.evaluate_energy(df_intake)
        # 3.2 评价函数：评价餐次比
        self.evaluate_meal_ratio(df_intake, df_meal)
        # 3.3 评价函数：评价非产能主要营养素
        self.evaluate_micronutrients(df_intake)
        # 4  评价函数: 评价一日食谱的宏量营养素供能比是否在推荐范围内
        self.evaluate_macro_ratios(df_intake)
        # 5  评价函数: 评价每餐的蛋白质氨基酸评分
        self.calculate_and_evaluate_per_meal_aas(df_meal)
        # 6 整体评价
        self.generate_overall_evaluation_and_suggestions()


e = Evaluation(df_man)
e.main()

--- 3.1 能量评价 ---
    实际值: 2771.98 kcal
    目标值: 2400.00 kcal
    实际值与目标值的差值占目标值的百分比: 15%
    评价: 每日能量摄入目标不在目标范围内 ❌

--- 3.2 餐次比评价 ---
  早餐 供能:
    实际供能: 804.54 kcal (实际功能占比: 29.02%)
    目标供能占比范围: 25% - 35%
    评价: 达标 ✅
  午餐 供能:
    实际供能: 1074.22 kcal (实际功能占比: 38.75%)
    目标供能占比范围: 30% - 40%
    评价: 达标 ✅
  晚餐 供能:
    实际供能: 893.23 kcal (实际功能占比: 32.22%)
    目标供能占比范围: 30% - 40%
    评价: 达标 ✅

--- 3.3 非产能主要营养素评价 ---
  钙 摄入量:
    实际摄入量: 412.50 mg
    目标量: 800.00 mg
    实际值占目标值的比例: 51.56%
    评价: 不足 ❌
  铁 摄入量:
    实际摄入量: 18.55 mg
    目标量: 12.00 mg
    实际值占目标值的比例: 154.58%
    评价: 达标或偏高 ✅
  锌 摄入量:
    实际摄入量: 11.34 mg
    目标量: 12.50 mg
    实际值占目标值的比例: 90.72%
    评价: 不足 ❌
  维生素A 摄入量:
    实际摄入量: 107.95 μg
    目标量: 800.00 μg
    实际值占目标值的比例: 13.49%
    评价: 不足 ❌
  维生素B1 摄入量:
    实际摄入量: 2.16 mg
    目标量: 1.40 mg
    实际值占目标值的比例: 154.00%
    评价: 达标或偏高 ✅
  维生素B2 摄入量:
    实际摄入量: 1.52 mg
    目标量: 1.40 mg
    实际值占目标值的比例: 108.50%
    评价: 达标或偏高 ✅
  维生素C 摄入量:
    实际摄入量: 42.16 mg
    目标量: 100.00 mg
    实际值占目标值的比例

In [26]:
# -*- coding: utf-8 -*-

import os
import numpy as np
import pandas as pd


# Assuming find_project_root, clean_food_code, clean_food_code_main functions are defined elsewhere and work.
# Assuming df_man_origin, df_woman_origin are loaded.
# Assuming df_man, df_woman are created as copies and have .name set (e.g., df_man.name = '男').


# --- Helper function to get major food group from code (used in Step 1) ---
def get_major_food_group_from_code(food_code, code_to_group_map):
    """
    根据食物编码的前两位查找对应的五大类别名称。
    非五大类别，则返回 '其他类别'。
    Assumes food_code is not NaN and is string-like.
    """
    if pd.isna(food_code) or not isinstance(food_code, str):
        return '其他类别'
    if len(food_code) < 2:  # Basic check if code has at least 2 characters
        return '其他类别'
    code = food_code[:2]
    return code_to_group_map.get(code, '其他类别')


# --- Helper function to count unique food types (used in Step 1) ---
def count_food_types(df: pd.DataFrame) -> int:
    """
    统计 DataFrame 中 '食物名称' 列的不重复数量。
    """
    if '食物名称' in df.columns:
        return df['食物名称'].nunique()
    return 0


# --- Evaluation Class (Modified to remove Fiber Energy and consolidate prints) ---

class Evaluation:
    def __init__(self, df: pd.DataFrame, display_flag: bool = False, rounding_decimals: int = 2):
        self.df = df.copy()
        self.display_flag = display_flag
        self.rounding_decimals = rounding_decimals

        # --- Standards (Comprehensive) ---
        self.standards = {
            'daily_variety_min': 12,  # 一日食谱种类最低数
            'energy_target': {'男': 2400, '女': 1900},  # 每日能量摄入目标 (kcal/d)
            'energy_acceptable_range_percent': 10,  # 能量适宜范围百分比 (例如 10 表示 ±10%)
            'meal_ratio_range': {'早餐': (0.25, 0.35), '午餐': (0.30, 0.40), '晚餐': (0.30, 0.40)},  # 餐次比推荐范围 (0-1)
            'micro_target': {  # 非产能营养素参考摄入量 (RNI/AI)
                '男': {'钙': 800, '铁': 12, '锌': 12.5, '维生素A': 800, '维生素B1': 1.4, '维生素B2': 1.4,
                       '维生素C': 100},
                '女': {'钙': 800, '铁': 20, '锌': 7.5, '维生素A': 700, '维生素B1': 1.2, '维生素B2': 1.2, '维生素C': 100}
            },
            'macro_ratio_range': {'蛋白质': (0.10, 0.15), '脂肪': (0.20, 0.30), '碳水化合物': (0.50, 0.65)},
            # 宏量营养素供能占比范围 (0-1)
            'energy_conversion': {'蛋白质': 4, '脂肪': 9, '碳水化合物': 4},  # 能量转换系数 (kcal/g) - 不包含膳食纤维
            'aas_ref_pattern': {  # AAS 参考模式 (mg/g 蛋白质)
                '异亮氨酸': 40, '亮氨酸': 70, '赖氨酸': 55, '含硫氨基酸': 35,
                '芳香族氨基酸': 60, '苏氨酸': 40, '色氨酸': 10, '缬氨酸': 50,
            },
            'aas_eval_criteria': {  # AAS 评价标准 (包含下限，不包含上限)
                '不合理': (0, 60), '不够合理': (60, 80), '比较合理': (80, 90), '合理': (90, float('inf')),
            },
            'five_major_food_names': ["谷、薯类", "蔬菜、菌藻、水果类", "畜、禽、鱼、蛋类及制品", "奶、干豆、坚果、种子类及制品",
                                      "植物油类"],  # 五大种类名称
            'code_prefix_to_major_group': self._build_code_to_group_map(),  # 食物编码前缀到五大类别的映射
            'nutrient_cols_per_100g': self._get_nutrient_cols_per_100g(df),  # 识别每100g营养成分列 (不含膳食纤维)
            'weight_col': '食物重量(克)',  # 标准重量列名
            'meal_col': '餐次',  # 标准餐次列名
        }

        # Initialize results dictionary
        self.evaluation_results = {
            'student_id': self.df.name if hasattr(self.df, 'name') and self.df.name is not None else '未知学生',
            'gender': self.df.name if hasattr(self.df, 'name') and self.df.name is not None else '未知',
        }

        # Ensure df has .name attribute
        if not hasattr(self.df, 'name') or self.df.name is None:
            self.df.name = '未知学生'

    def _build_code_to_group_map(self):
        """ 构建食物编码前缀到五大类别的映射字典。"""
        # Define mapping source data (should match standards)
        five_major_food_groups_source = [
            {"谷类及制品": "01", "薯类、淀粉及制品": "02"},
            {"蔬菜类及制品": "04", "菌藻类": "05", "水果类及制品": "06"},
            {"畜肉类及制品": "08", "禽肉类及制品": "09", "鱼虾蟹贝类": "12", "蛋类及制品": "11"},
            {"乳类及制品": "10", "干豆类及制品": "03", "坚果、种子类": "07"},
            {"植物油类": "18"}
        ]
        five_major_food_names_source = ["谷、薯类", "蔬菜、菌藻、水果类", "畜、禽、鱼、蛋类及制品",
                                        "奶、干豆、坚果、种子类及制品", "植物油类"]

        code_to_group_map = {}
        for i, group_codes_dict in enumerate(five_major_food_groups_source):
            group_name = five_major_food_names_source[i]
            for sub_group_name, code_prefix in group_codes_dict.items():
                code_to_group_map[code_prefix] = group_name
        return code_to_group_map

    def _get_nutrient_cols_per_100g(self, df: pd.DataFrame) -> list[str]:
        """
        从 DataFrame 的列名中识别出每100g营养成分列。
        不包含膳食纤维。
        """
        nutrient_patterns = {
            '碳水化合物': 'g/100g', '蛋白质': 'g/100g', '脂肪': 'g/100g',
            # '膳食纤维': 'g/100g', # Excluded as per user request
            '钙': 'mg/100g', '铁': 'mg/100g', '锌': 'mg/100g',
            '维生素A': 'μg/100g', '维生素B1': 'mg/100g', '维生素B2': 'mg/100g', '维生素C': 'mg/100g',
            '异亮氨酸': 'g/100g', '亮氨酸': 'g/100g', '赖氨酸': 'g/100g',
            '含硫氨基酸': 'g/100g', '芳香族氨基酸': 'g/100g',
            '苏氨酸': 'g/100g', '色氨酸': 'g/100g', '缬氨酸': 'g/100g',
            # Add other relevant nutrients if in your data
        }

        nutrient_cols = []
        for name, unit_pattern in nutrient_patterns.items():
            col_name = f'{name} ({unit_pattern})'
            if col_name in df.columns:
                nutrient_cols.append(col_name)

        if not nutrient_cols and self.display_flag:
            print("警告: 未能从DataFrame列名中识别出标准格式的每100g营养成分列。计算可能失败。")

        return nutrient_cols

    #  --- 1 分析食物结构 ---
    def analyze_food_structure(self):
        """ 分析食谱中的食物结构，结果存储在 self.evaluation_results['food_structure']。 """
        df = self.df
        standards = self.standards
        student_id = self.evaluation_results['student_id']

        # print(f"\n--- {student_id} - 1 分析食物结构 ---") # Printing moved to Step 6 summary

        if '食物类别' not in df.columns:
            df['食物类别'] = df['食物编码'].apply(
                lambda x: get_major_food_group_from_code(x, standards['code_prefix_to_major_group'])
            )

        categories_present = df[df['食物类别'] != '其他类别']['食物类别'].unique().tolist()
        missing_categories = [cat for cat in standards['five_major_food_names'] if cat not in categories_present]
        all_five_present = len(missing_categories) == 0

        unique_food_types_count = count_food_types(df)
        daily_variety_met = unique_food_types_count > standards['daily_variety_min']

        other_category_items = df[df['食物类别'] == '其他类别']['主要成分'].unique().tolist()

        # Optional: Food quantities summary
        food_quantities_summary = None
        weight_col = standards.get('weight_col', '食物重量(克)')
        if '食物名称' in df.columns and weight_col in df.columns:
            food_quantities_summary_df = df.groupby('食物名称')[weight_col].sum().reset_index()
            food_quantities_summary = food_quantities_summary_df.to_dict('records')

        self.evaluation_results['food_structure'] = {
            'categories_present': categories_present,
            'missing_categories': missing_categories,
            'all_five_present': all_five_present,
            'unique_food_types_count': unique_food_types_count,
            'daily_variety_met': daily_variety_met,
            'other_category_items': other_category_items,
            'food_quantities_summary': food_quantities_summary,
        }
        # print(f"--- {student_id} 食物结构分析完成 ---") # Printing moved to Step 6 summary

    #  --- 2 计算主要营养素含量 ---
    def calculate_nutrient_intakes(self) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        计算日食谱的日总和每餐总营养素摄入量及能量。
        结果存储在 self.evaluation_results['nutrient_intake']。
        不包含膳食纤维的能量计算。

        Returns:
            一个元组，包含：
            - df_intake: DataFrame（一日各营养素摄入量及能量总量，1行）
            - df_meal: DataFrame（餐次的各营养素摄入量及能量总量）
        """
        df = self.df
        student_id = self.evaluation_results['student_id']

        weight_col = self.standards.get('weight_col', '食物重量(克)')
        meal_col = self.standards.get('meal_col', '餐次')
        energy_conversion = self.standards.get('energy_conversion', {})  # Does not include Fiber
        nutrient_cols_per_100g = self.standards.get('nutrient_cols_per_100g', [])  # Should not include Fiber

        if weight_col not in df.columns:
            # print(f"警告: 未找到重量列 '{weight_col}', 无法计算摄入量.") # Printing moved to Step 6 summary if needed
            df[weight_col] = df['可食部（克/份）'].fillna(0) * df['食用份数'].fillna(0)

        # --- 2.1 计算每行营养素摄入量 ---
        # print(f"\n--- {student_id} - 2 计算主要营养素含量 ---") # Printing moved to Step 6 summary
        # print(f"--- 2.1 计算 {student_id} 食谱的主要营养素摄入量 ---") # Printing moved to Step 6 summary

        intake_cols = []
        for nutrient_col_100g in nutrient_cols_per_100g:
            parts = nutrient_col_100g.replace(')', '').split('(')
            if len(parts) == 2:
                nutrient_name = parts[0].strip()
                unit_info = parts[1].strip()
                intake_unit = unit_info.split('/')[0].strip()
                intake_col_name = f'{nutrient_name}摄入量 ({intake_unit})'

                if nutrient_col_100g in df.columns and weight_col in df.columns:
                    df[intake_col_name] = (df[weight_col].fillna(0) / 100) * df[nutrient_col_100g].fillna(0)
                    intake_cols.append(intake_col_name)
                # else: Optional warning if a standard nutrient col is missing

        # --- Calculate per-row energy contribution from macros (P, F, C) ---
        per_row_energy_contribution_cols = []
        # Explicitly list macro names for energy calculation based on energy_conversion
        macro_names_for_energy = ['蛋白质', '脂肪', '碳水化合物']  # Based on energy_conversion keys
        for substance in macro_names_for_energy:
            intake_col_g = f'{substance}摄入量 (g)'  # Assuming intake columns for macros are in grams
            energy_col_name = f'{substance}能量贡献 (kcal)'
            if intake_col_g in df.columns and energy_conversion.get(substance) is not None:
                df[energy_col_name] = df[intake_col_g].fillna(0) * energy_conversion.get(substance, 0)
                per_row_energy_contribution_cols.append(energy_col_name)
            # else: Optional warning if macro intake col or conversion is missing

        # --- 2.2 计算一日总营养素摄入量 ---
        # print(f"\n--- 2.2 计算 {student_id} 一日总营养素摄入量 ---") # Printing moved to Step 6 summary

        existing_intake_cols_for_daily = [col for col in intake_cols if col in df.columns]
        daily_nutrient_intake_dict = {}
        if existing_intake_cols_for_daily:
            for intake_col in existing_intake_cols_for_daily:
                daily_nutrient_intake_dict[intake_col] = df[intake_col].sum()

        # --- 2.3 计算总能量 (日总) ---
        # Calculate total daily energy ONLY from P, F, C intake grams
        protein_total_g = daily_nutrient_intake_dict.get('蛋白质摄入量 (g)', 0)
        fat_total_g = daily_nutrient_intake_dict.get('脂肪摄入量 (g)', 0)
        carb_total_g = daily_nutrient_intake_dict.get('碳水化合物摄入量 (g)', 0)
        # Fiber intake is not used for total energy calculation here

        total_calculated_energy_kcal = (protein_total_g * energy_conversion.get('蛋白质', 0) +
                                        fat_total_g * energy_conversion.get('脂肪', 0) +
                                        carb_total_g * energy_conversion.get('碳水化合物', 0))

        daily_nutrient_intake_dict['总能量摄入量 (kcal)'] = total_calculated_energy_kcal

        df_intake = pd.DataFrame([daily_nutrient_intake_dict])

        # print(f"--- {student_id} 完成计算一日总营养素和能量摄入量 ---") # Printing moved to Step 6 summary

        # --- 2.4 计算每餐总营养素摄入量及能量 ---
        # Sum calculated columns per meal. This includes '摄入量 (...)' and '...能量贡献 (kcal)' for P,F,C.
        # print(f"--- 2.4 计算 {student_id} 每餐总营养素摄入量及能量 ---") # Printing moved to Step 6 summary

        # Identify ALL columns that are results of calculation and should be summed per meal
        # This is the combined list of intake_cols and per_row_energy_contribution_cols
        cols_to_sum_per_meal = intake_cols + per_row_energy_contribution_cols

        existing_cols_to_sum_per_meal = [col for col in cols_to_sum_per_meal if col in df.columns]

        if meal_col not in df.columns:
            df_meal = pd.DataFrame()
            # print(f"错误：缺少餐次列 '{meal_col}'，无法计算每餐总量。") # Printing moved to Step 6 summary
        elif df.empty:
            df_meal = pd.DataFrame()
            # print("警告：输入DataFrame为空，无法计算每餐总量。") # Printing moved to Step 6 summary
        elif not existing_cols_to_sum_per_meal:
            df_meal = pd.DataFrame()
            # print("警告：没有找到任何需要按餐次求和的摄入量或能量贡献列。无法计算每餐总量。") # Printing moved to Step 6 summary
        else:
            df_meal = df.groupby(meal_col)[existing_cols_to_sum_per_meal].sum()

            # Add the '总能量摄入量 (kcal)' column to df_meal by summing the per-meal energy contribution columns (P,F,C)
            existing_per_meal_energy_cols = [col for col in per_row_energy_contribution_cols if col in df_meal.columns]
            if existing_per_meal_energy_cols:
                df_meal['总能量摄入量 (kcal)'] = df_meal[existing_per_meal_energy_cols].sum(axis=1)
            else:
                df_meal['总能量摄入量 (kcal)'] = 0  # Set to 0 if no energy contribution columns were summed

        # print(f"--- {student_id} 完成计算每餐总营养素和能量摄入量 ---") # Printing moved to Step 6 summary

        self.evaluation_results['nutrient_intake'] = {
            'daily_totals_df': df_intake,
            'meal_totals_df': df_meal
        }

        return df_intake, df_meal

    #  --- 3 评价各项指标 ---

    # --- 3.1 评价能量 ---
    # Modified logic for percentage and range check, removed printing
    def evaluate_energy(self, df_intake: pd.DataFrame) -> None:
        """
        评价一日总能量摄入量是否符合个体需要 (根据附件4评价原则1)。
        结果存储在 self.evaluation_results['evaluation_energy']。
        """
        gender = self.evaluation_results['gender']
        rounding_decimals = self.rounding_decimals

        energy_evaluation = {}

        actual_energy_kcal = df_intake.get('总能量摄入量 (kcal)', pd.Series([0])).values[0]
        target_energy_kcal = self.standards['energy_target'].get(gender)
        acceptable_range_percent = self.standards.get('energy_acceptable_range_percent')

        energy_evaluation['actual_kcal'] = round(actual_energy_kcal, rounding_decimals)
        energy_evaluation['target_kcal'] = target_energy_kcal

        comment = "标准缺失或数据缺失"
        percentage_of_target = None
        acceptable_range_kcal = None

        if target_energy_kcal is not None and target_energy_kcal > 0:
            percentage_of_target = (actual_energy_kcal / target_energy_kcal) * 100

            if acceptable_range_percent is not None and isinstance(acceptable_range_percent, (int, float)):
                lower_bound = target_energy_kcal * (1 - acceptable_range_percent / 100)
                upper_bound = target_energy_kcal * (1 + acceptable_range_percent / 100)
                acceptable_range_kcal = (round(lower_bound, 2), round(upper_bound, 2))

                if lower_bound <= actual_energy_kcal <= upper_bound:
                    comment = "适宜"
                elif actual_energy_kcal < lower_bound:
                    comment = "偏低"
                else:  # actual_energy_kcal > upper_bound
                    comment = "偏高"
            else:
                # Fallback evaluation if range percentage is missing
                comment = "适宜 (范围标准缺失)" if actual_energy_kcal >= target_energy_kcal else "偏低 (范围标准缺失)"

        elif target_energy_kcal == 0:
            comment = "目标为零，无法评价"

        energy_evaluation['percentage_of_target'] = round(percentage_of_target,
                                                          rounding_decimals) if percentage_of_target is not None else None
        energy_evaluation['acceptable_range_kcal'] = acceptable_range_kcal
        energy_evaluation['comment'] = comment

        self.evaluation_results['evaluation_energy'] = energy_evaluation

    # --- 3.2 评价餐次比 ---
    # Removed print statements
    def evaluate_meal_ratio(self, df_intake: pd.DataFrame, df_meal: pd.DataFrame) -> None:
        """
        评价三餐供能比是否在推荐范围内 (根据附件4评价原则2)。
        结果存储在 self.evaluation_results['evaluation_meal_ratio']。
        """
        rounding_decimals = self.rounding_decimals
        meal_ratio_evaluation = {}

        meal_energy_col = '总能量摄入量 (kcal)'
        total_daily_energy = df_intake.get(meal_energy_col, pd.Series([0])).values[0]

        meal_ratio_ranges = self.standards.get('meal_ratio_range')

        meal_ratio_evaluation['meal_ratios'] = {}
        overall_comment = "达标 ✅"  # Optimistic initialization

        meal_order = ['早餐', '午餐', '晚餐']

        if total_daily_energy <= 0 or meal_energy_col not in df_meal.columns or df_meal.empty or meal_ratio_ranges is None:
            overall_comment = "总能量为零、每餐能量数据缺失或标准缺失，无法评价"
        else:
            for meal in meal_order:
                meal_energy = df_meal.loc[
                    meal, meal_energy_col] if meal in df_meal.index and meal_energy_col in df_meal.columns else 0
                meal_percentage = (meal_energy / total_daily_energy) if total_daily_energy > 0 else 0
                target_range = meal_ratio_ranges.get(meal)

                meal_eval = {
                    'actual_kcal': round(meal_energy, rounding_decimals),
                    'actual_percentage': round(meal_percentage * 100, rounding_decimals),
                    'target_range_percent': None,
                    'comment': "标准缺失或数据缺失"
                }
                meal_ratio_evaluation['meal_ratios'][meal] = meal_eval

                if target_range is not None and isinstance(target_range, tuple) and len(
                        target_range) == 2 and isinstance(target_range[0], (int, float)) and isinstance(target_range[1],
                                                                                                        (int, float)):
                    target_min_percent = target_range[0] * 100
                    target_max_percent = target_range[1] * 100
                    meal_eval['target_range_percent'] = (round(target_min_percent, 2), round(target_max_percent, 2))

                    if target_range[0] <= meal_percentage <= target_range[1]:
                        meal_eval['comment'] = "达标"
                    else:
                        meal_eval['comment'] = "偏低" if meal_percentage < target_range[0] else "偏高"
                        if overall_comment == "达标 ✅":
                            overall_comment = "部分餐次比偏离"
                else:
                    if overall_comment == "达标 ✅":
                        overall_comment = "部分餐次比标准问题"

        meal_ratio_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_meal_ratio'] = meal_ratio_evaluation

    # --- 3.3 评价非产能主要营养素 ---

    def evaluate_micronutrients(self, df_intake: pd.DataFrame) -> None:
        """
        评价非产能主要营养素摄入量是否达到标准 (根据附件4评价原则3)。
        结果存储在 self.evaluation_results['evaluation_micronutrients']。
        """
        gender = self.evaluation_results['gender']
        rounding_decimals = self.rounding_decimals
        micronutrients_evaluation = {}

        micronutrient_targets = self.standards['micro_target'].get(gender)

        micronutrients_evaluation['micronutrients'] = {}
        overall_comment = "基本达标 😊"

        if micronutrient_targets is None or not isinstance(micronutrient_targets, dict) or not micronutrient_targets:
            overall_comment = "标准缺失，无法评价"
        else:
            micros_to_evaluate = list(micronutrient_targets.keys())

            for micro_name in micros_to_evaluate:
                target_amount = micronutrient_targets.get(micro_name)

                actual_intake_col_name = next(
                    (col for col in df_intake.columns if col.startswith(f'{micro_name}摄入量 (')),
                    None
                )
                actual_intake = df_intake[actual_intake_col_name].values[
                    0] if actual_intake_col_name in df_intake.columns and not df_intake.empty else 0

                micro_eval = {
                    'actual_intake': None,
                    'target_amount': target_amount,
                    'unit': None,
                    'percentage_of_rni_ai': None,
                    'comment': "计算失败或标准缺失"
                }
                micronutrients_evaluation['micronutrients'][micro_name] = micro_eval

                if target_amount is None or not isinstance(target_amount, (int, float)):
                    micro_eval['comment'] = "标准缺失或格式错误"
                    if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素标准问题"
                elif actual_intake_col_name is None:
                    micro_eval['comment'] = "数据列缺失"
                    if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素数据列缺失"
                    overall_comment = "部分未达标，需改进"
                else:
                    micro_eval['actual_intake'] = round(actual_intake, rounding_decimals)
                    actual_unit_match = actual_intake_col_name.split('(')[-1].replace(')',
                                                                                      '').strip() if '(' in actual_intake_col_name else '?'
                    micro_eval['unit'] = actual_unit_match

                    if target_amount > 0:
                        percentage = (actual_intake / target_amount) * 100
                        micro_eval['percentage_of_rni_ai'] = round(percentage, rounding_decimals)

                        if percentage >= 100:
                            micro_eval['comment'] = "达标或偏高"
                        else:
                            micro_eval['comment'] = "不足"
                            if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素摄入不足"
                            overall_comment = "部分未达标，需改进"
                    else:
                        micro_eval['comment'] = "目标为零，无法评价"
                        if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素标准为零"
                        overall_comment = "部分未达标，需改进"

        micronutrients_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_micronutrients'] = micronutrients_evaluation

    # --- 4 评价宏量营养素供能比 ---
    # Removed print statements, adjusted for no fiber energy
    def evaluate_macro_ratios(self, df_intake: pd.DataFrame) -> None:
        """
        评价一日食谱的宏量营养素供能比是否在推荐范围内 (根据附件4评价原则4)。
        结果存储在 self.evaluation_results['evaluation_macro_ratios']。
        能量计算不包含膳食纤维。
        """
        rounding_decimals = self.rounding_decimals
        macro_ratios_evaluation = {}

        # Get total daily energy and macro grams (use .get for safety)
        # Total daily energy is calculated in Step 2 WITHOUT fiber
        total_daily_energy = df_intake.get('总能量摄入量 (kcal)', pd.Series([0])).values[0]
        protein_g = df_intake.get('蛋白质摄入量 (g)', pd.Series([0])).values[0]
        fat_g = df_intake.get('脂肪摄入量 (g)', pd.Series([0])).values[0]
        carb_g = df_intake.get('碳水化合物摄入量 (g)', pd.Series([0])).values[0]
        # Fiber intake is not used for energy calculation or macro ratios here

        macro_ratio_ranges = self.standards.get('macro_ratio_range')
        energy_conversion = self.standards.get('energy_conversion', {})  # Does not include Fiber

        macro_ratios_evaluation['macro_ratios'] = {}
        overall_comment = "达标 ✅"  # Optimistic initialization

        # --- Safety Check ---
        if total_daily_energy <= 0 or macro_ratio_ranges is None or not isinstance(macro_ratio_ranges,
                                                                                   dict) or not macro_ratio_ranges or energy_conversion is None or not isinstance(
            energy_conversion, dict) or not energy_conversion:
            overall_comment = "日总能量为零、标准缺失或格式不正确，无法评价"
        else:
            # Calculate energy from each macro (kcal) using conversion factors
            protein_kcal = protein_g * energy_conversion.get('蛋白质', 0)
            fat_kcal = fat_g * energy_conversion.get('脂肪', 0)
            carb_kcal = carb_g * energy_conversion.get('碳水化合物', 0)

            macro_energy_dict = {'蛋白质': protein_kcal, '脂肪': fat_kcal, '碳水化合物': carb_kcal}

            # --- Evaluate Each Macro ---
            macros_to_evaluate = ['蛋白质', '脂肪', '碳水化合物']
            for macro_name in macros_to_evaluate:
                actual_kcal = macro_energy_dict.get(macro_name, 0)
                # Calculate actual ratio relative to total daily energy (P+F+C energy sum)
                actual_ratio = (actual_kcal / total_daily_energy) if total_daily_energy > 0 else 0  # Ratio in 0-1 range
                target_range = macro_ratio_ranges.get(macro_name)  # Range in 0-1 range

                macro_eval = {
                    'actual_kcal': round(actual_kcal, rounding_decimals),
                    'actual_ratio': round(actual_ratio, rounding_decimals),
                    'actual_percentage': round(actual_ratio * 100, rounding_decimals),  # Percentage 0-100
                    'target_range': target_range,
                    'target_range_percent': None,
                    'comment': "标准缺失或格式错误"
                }
                macro_ratios_evaluation['macro_ratios'][macro_name] = macro_eval

                if target_range is not None and isinstance(target_range, tuple) and len(
                        target_range) == 2 and isinstance(target_range[0], (int, float)) and isinstance(target_range[1],
                                                                                                        (int, float)):
                    target_min_percent = target_range[0] * 100
                    target_max_percent = target_range[1] * 100
                    macro_eval['target_range_percent'] = (round(target_min_percent, 2), round(target_max_percent, 2))

                    if target_range[0] <= actual_ratio <= target_range[1]:
                        macro_eval['comment'] = "达标"
                    else:
                        macro_eval['comment'] = "偏低" if actual_ratio < target_range[0] else "偏高"
                        if overall_comment == "达标 ✅":
                            overall_comment = "部分宏量比偏离"
                else:
                    if overall_comment == "达标 ✅":
                        overall_comment = "部分宏量比标准问题"

            # --- Check consistency of P+F+C energy sum vs total daily energy ---
            # Compare the sum of P+F+C kcal (calculated here) to the total daily energy (calculated in Step 2)
            sum_pfc_kcal = protein_kcal + fat_kcal + carb_kcal
            if total_daily_energy > 0 and abs(sum_pfc_kcal - total_daily_energy) / total_daily_energy > 0.01:
                if overall_comment == "达标 ✅":
                    overall_comment += " (计算警告: PFC能量总和与日总量不符)"

        macro_ratios_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_macro_ratios'] = macro_ratios_evaluation

    # --- 5 计算并评价每餐的蛋白质氨基酸评分 (AAS) ---
    # Removed print statements
    def calculate_and_evaluate_per_meal_aas(self, df_meal: pd.DataFrame) -> None:
        """
        计算并评价每餐的蛋白质氨基酸评分 (AAS) (根据附件4评价原则5)。
        结果存储在 self.evaluation_results['evaluation_aas']。
        """
        rounding_decimals = self.rounding_decimals
        aas_evaluation = {}

        aas_ref_pattern = self.standards.get('aas_ref_pattern')
        aas_eval_criteria = self.standards.get('aas_eval_criteria')

        aas_evaluation['meal_aas'] = {}
        overall_comment = "达标 ✅"  # Optimistic initialization (if all meals are '合理' or '比较合理')

        # --- Safety Check ---
        if aas_ref_pattern is None or not isinstance(aas_ref_pattern, dict) or not aas_ref_pattern:
            overall_comment = "AAS 参考模式标准缺失或格式不正确，无法评价"
        elif aas_eval_criteria is None or not isinstance(aas_eval_criteria, dict) or not aas_eval_criteria:
            overall_comment = "AAS 评价标准缺失或格式不正确，无法评价"
        elif df_meal.empty:
            overall_comment = "每餐数据为空，无法计算 AAS"
        else:
            # Identify required columns (Protein in g, AA intake in g or mg)
            protein_intake_col_g = '蛋白质摄入量 (g)'
            essential_aa_names = list(aas_ref_pattern.keys())
            essential_aa_intake_cols = []  # List to store actual AA intake column names in df_meal

            # Check for AA intake columns (prioritize mg, then g)
            for aa_name in essential_aa_names:
                col_mg = f'{aa_name}摄入量 (mg)'
                col_g = f'{aa_name}摄入量 (g)'
                if col_mg in df_meal.columns:
                    essential_aa_intake_cols.append(col_mg)
                elif col_g in df_meal.columns:
                    essential_aa_intake_cols.append(col_g)
                # else: Optional warning if AA intake col is missing

            # Check if protein column exists and if we found intake columns for all essential AAs
            if protein_intake_col_g not in df_meal.columns:
                overall_comment = f"缺少蛋白质摄入量列 '{protein_intake_col_g}'，无法计算 AAS"
            elif len(essential_aa_intake_cols) < len(essential_aa_names):
                # Need to identify which ones are missing if overall_comment is used for this
                missing_aa_cols = [f'{aa_name}摄入量 (g/mg)' for aa_name in essential_aa_names if
                                   f'{aa_name}摄入量 (mg)' not in essential_aa_intake_cols and f'{aa_name}摄入量 (g)' not in essential_aa_intake_cols]
                overall_comment = f"缺少部分必需氨基酸摄入量列 ({', '.join(missing_aa_cols)})，无法计算 AAS"
            else:
                # --- Calculate and Evaluate AAS for Each Meal ---
                meal_order = ['早餐', '午餐', '晚餐']

                overall_aas_sufficient = True

                for meal in meal_order:
                    meal_eval_default = {'comment': '数据缺失', 'aas': None, 'limiting_aa': None, 'protein_g': None}

                    if meal not in df_meal.index:
                        aas_evaluation['meal_aas'][meal] = meal_eval_default
                        overall_aas_sufficient = False
                        continue

                    meal_protein_g = df_meal.loc[
                        meal, protein_intake_col_g] if protein_intake_col_g in df_meal.columns else 0

                    if meal_protein_g <= 0:
                        meal_eval_default['comment'] = '蛋白质摄入为零，无法计算 AAS'
                        aas_evaluation['meal_aas'][meal] = meal_eval_default
                        overall_aas_sufficient = False
                        continue

                    # --- Calculate AAS for this meal ---
                    limiting_aa = None
                    min_ratio = float('inf')
                    aa_ratios = {}

                    calculation_successful = True

                    for aa_intake_col in essential_aa_intake_cols:
                        aa_name = aa_intake_col.split('摄入量')[0].strip()
                        aa_unit = aa_intake_col.split('(')[-1].replace(')', '').strip()

                        aa_intake_amount = df_meal.loc[meal, aa_intake_col] if aa_intake_col in df_meal.columns else 0

                        aa_intake_mg = aa_intake_amount * 1000 if aa_unit.lower() == 'g' else aa_intake_amount

                        ref_amount_mg_per_g_protein = aas_ref_pattern.get(aa_name)

                        if ref_amount_mg_per_g_protein is None or ref_amount_mg_per_g_protein <= 0:
                            calculation_successful = False
                            break

                        ratio = aa_intake_mg / (meal_protein_g * ref_amount_mg_per_g_protein) if (
                                meal_protein_g > 0 and ref_amount_mg_per_g_protein > 0) else 0
                        aa_ratios[aa_name] = round(ratio, rounding_decimals)

                        if ratio < min_ratio:
                            min_ratio = ratio
                            limiting_aa = aa_name

                    # --- Store and Evaluate AAS Score for this meal ---
                    meal_eval = {
                        'protein_g': round(meal_protein_g, rounding_decimals),
                        'aas': None,
                        'limiting_aa': None,
                        'comment': "计算失败",
                    }

                    if calculation_successful and min_ratio != float('inf'):
                        aas_score = round(min_ratio * 100, rounding_decimals)
                        meal_eval['aas'] = aas_score
                        meal_eval['limiting_aa'] = limiting_aa

                        meal_aas_comment = "未知等级"
                        is_current_meal_sufficient = False
                        sorted_criteria = sorted(aas_eval_criteria.items(), key=lambda item: item[1][0])

                        for comment, range_tuple in sorted_criteria:
                            if range_tuple[0] <= aas_score < range_tuple[1]:
                                meal_aas_comment = comment
                                if comment in ['合理', '比较合理']:
                                    is_current_meal_sufficient = True
                                break

                        meal_eval['comment'] = meal_aas_comment

                        if not is_current_meal_sufficient:
                            overall_aas_sufficient = False

                    else:
                        overall_aas_sufficient = False

                    aas_evaluation['meal_aas'][meal] = meal_eval

                # --- Determine Overall AAS Comment ---
                if overall_aas_sufficient:
                    overall_comment = "所有餐次 AAS 达标 ✅"
                else:
                    overall_comment = "部分餐次 AAS 不足或计算/数据缺失 ❌"

        aas_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_aas'] = aas_evaluation

    # --- 6 综合所有评价结果，生成整体评价和膳食建议 ---
    def generate_overall_evaluation_and_suggestions(self) -> str:
        """
        综合所有评价结果，生成整体评价和膳食建议。
        打印报告到控制台。
        """
        evaluation_results = self.evaluation_results
        standards = self.standards
        student_id = evaluation_results.get('student_id', '未知学生')
        gender = evaluation_results.get('gender', '未知')
        rounding_decimals = self.rounding_decimals

        overall_summary = f"\n--- {student_id} ({gender}) 食谱整体评价与建议 ---\n"

        overall_summary += "\n各项评价总结:\n"

        # 1. 食物结构总结
        fs_eval = evaluation_results.get('food_structure', {})
        overall_summary += "  食物结构:\n"
        if fs_eval:
            overall_summary += f"    种类数量: {fs_eval.get('unique_food_types_count', 'N/A')} 种 (目标 > {standards.get('daily_variety_min', 'N/A')} 种) - 评价: {'达标 ✅' if fs_eval.get('daily_variety_met', False) else '不足 ❌'}\n"
            overall_summary += f"    五大类包含: {'齐全 ✅' if fs_eval.get('all_five_present', False) else '不齐全 ❌'}\n"
            if fs_eval.get('missing_categories'):
                overall_summary += f"      缺少的类别: {', '.join(fs_eval['missing_categories'])}\n"
            if fs_eval.get('other_category_items'):
                overall_summary += f"    未归类食物: {', '.join(fs_eval['other_category_items'])}\n"
        else:
            overall_summary += "    食物结构评价数据缺失。\n"

        # 3.1 能量摄入总结
        energy_eval = evaluation_results.get('evaluation_energy', {})
        overall_summary += "  能量摄入:\n"
        if energy_eval:
            actual_energy_kcal = energy_eval.get('actual_kcal', 'N/A')
            target_energy_kcal = energy_eval.get('target_kcal')
            percentage = energy_eval.get('percentage_of_target', 'N/A')
            acceptable_range_kcal = energy_eval.get('acceptable_range_kcal', 'N/A')

            summary_line = f"    总能量: {actual_energy_kcal:.{rounding_decimals}f} kcal"
            if target_energy_kcal is not None:
                summary_line += f" (目标 {target_energy_kcal:.0f} kcal"
                if acceptable_range_kcal != 'N/A':
                    summary_line += f", 适宜范围 {acceptable_range_kcal[0]:.0f}-{acceptable_range_kcal[1]:.0f} kcal"
                summary_line += f", 占目标 {percentage:.1f}%)"
            summary_line += f" - 评价: {energy_eval.get('comment', 'N/A')}\n"
            overall_summary += summary_line
        else:
            overall_summary += "    能量评价数据缺失。\n"

        # 3.2 餐次比总结
        meal_ratio_eval = evaluation_results.get('evaluation_meal_ratio', {})
        overall_summary += "  餐次供能比:\n"
        if meal_ratio_eval and meal_ratio_eval.get('meal_ratios'):
            for meal, eval_data in meal_ratio_eval['meal_ratios'].items():
                summary_line = f"    {meal}: {eval_data.get('actual_percentage', 'N/A'):.1f}%"
                if eval_data.get('target_range_percent'):
                    summary_line += f" (目标 {eval_data['target_range_percent'][0]:.0f}%-{eval_data['target_range_percent'][1]:.0f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  餐次比整体评价: {meal_ratio_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    餐次比评价数据缺失。\n"

        # 4. 宏量营养素供能比总结
        macro_eval = evaluation_results.get('evaluation_macro_ratios', {})
        overall_summary += "  宏量供能比:\n"
        if macro_eval and macro_eval.get('macro_ratios'):
            for macro, eval_data in macro_eval['macro_ratios'].items():
                summary_line = f"    {macro}: {eval_data.get('actual_percentage', 'N/A'):.1f}%"
                if eval_data.get('target_range_percent'):
                    summary_line += f" (目标 {eval_data['target_range_percent'][0]:.0f}%-{eval_data['target_range_percent'][1]:.0f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  宏量供能比整体评价: {macro_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    宏量营养素供能比评价数据缺失。\n"

        # 3.3 非产能主要营养素总结
        micro_eval = evaluation_results.get('evaluation_micronutrients', {})
        overall_summary += "  非产能营养素:\n"
        if micro_eval and micro_eval.get('micronutrients'):
            for micro, eval_data in micro_eval['micronutrients'].items():
                summary_line = f"    {micro}: {eval_data.get('actual_intake', 'N/A'):.{rounding_decimals}f}{eval_data.get('unit', '?')}"
                if eval_data.get('target_amount') is not None:
                    summary_line += f" (目标 {eval_data['target_amount']:.0f}{eval_data.get('unit', '?')}, 占目标 {eval_data.get('percentage_of_rni_ai', 'N/A'):.1f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  非产能营养素整体评价: {micro_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    非产能主要营养素评价数据缺失。\n"

        # 5. AAS 总结
        aas_eval = evaluation_results.get('evaluation_aas', {})
        overall_summary += "  每餐 AAS:\n"
        if aas_eval and aas_eval.get('meal_aas'):
            for meal, eval_data in aas_eval['meal_aas'].items():
                summary_line = f"    {meal}: AAS {eval_data.get('aas', 'N/A'):.1f}"
                if eval_data.get('limiting_aa'):
                    summary_line += f" (限制性氨基酸: {eval_data['limiting_aa']})"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  每餐 AAS 整体评价: {aas_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    每餐 AAS 评价数据缺失或计算失败。\n"

        # --- Comprehensive Evaluation Conclusions and Dietary Suggestions ---
        overall_summary += "\n综合评价结论:\n"
        suggestions = []

        # Food Structure Suggestions
        if fs_eval and not fs_eval.get('daily_variety_met', False):
            suggestions.append(
                f"食谱种类不足，建议增加每日食物种类数量，目标 > {standards.get('daily_variety_min', 'N/A')} 种，以增加食物多样性。")
        if fs_eval and not fs_eval.get('all_five_present', False):
            suggestions.append(
                f"食谱包含的食物类别不全，建议增加 {', '.join(fs_eval.get('missing_categories', []))} 等五大类食物的摄入。")

        # Energy Suggestions
        energy_comment = energy_eval.get('comment')
        if energy_comment == '偏低':
            suggestions.append(
                f"总能量摄入偏低 ({energy_eval.get('actual_kcal', 'N/A'):.0f} kcal)，建议适量增加食物摄入总量。")
        elif energy_comment == '偏高':
            suggestions.append(
                f"总能量摄入偏高 ({energy_eval.get('actual_kcal', 'N/A'):.0f} kcal)，建议适量减少食物摄入总量。")

        # Meal Ratio Suggestions
        if meal_ratio_eval and meal_ratio_eval.get('overall_comment') == '部分餐次比偏离':
            for meal, eval_data in meal_ratio_eval.get('meal_ratios', {}).items():
                meal_comment = eval_data.get('comment')
                if meal_comment == '偏低':
                    suggestions.append(
                        f"{meal} 供能比偏低 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议增加 {meal} 的食物摄入量，以使供能更均衡。")
                elif meal_comment == '偏高':
                    suggestions.append(
                        f"{meal} 供能比偏高 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议减少 {meal} 的食物摄入量，并合理分配到其他餐次。")

        # Macro Ratio Suggestions
        if macro_eval and macro_eval.get('overall_comment') == '部分宏量比偏离':
            for macro, eval_data in macro_eval.get('macro_ratios', {}).items():
                macro_comment = eval_data.get('comment')
                if macro_comment == '偏低':
                    suggestions.append(
                        f"{macro} 供能比偏低 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议增加富含 {macro} 的食物摄入，如{'全谷物、薯类' if macro == '碳水化合物' else ('鱼禽蛋瘦肉、豆制品' if macro == '蛋白质' else '优质植物油、坚果')}等。")
                elif macro_comment == '偏高':
                    suggestions.append(
                        f"{macro} 供能比偏高 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议减少富含 {macro} 的食物摄入。")

        # Micronutrient Suggestions (focus on '不足')
        if micro_eval and micro_eval.get('overall_comment') in ['部分非产能营养素摄入不足', '部分未达标，需改进']:
            suggestions.append("食谱中部分非产能主要营养素摄入不足，需重点改进：")
            for micro, eval_data in micro_eval.get('micronutrients', {}).items():
                micro_comment = eval_data.get('comment')
                if micro_comment in ['不足', '严重不足']:
                    suggestion_text = f"  {micro}: 摄入量不足 (占目标 {eval_data.get('percentage_of_rni_ai', 'N/A'):.1f}%)，建议增加富含 {micro} 的食物。"
                    if micro == '钙':
                        suggestion_text += " (如奶制品、豆制品、深绿色蔬菜)"
                    elif micro == '铁':
                        suggestion_text += " (如瘦肉、动物肝脏、木耳)"
                    elif micro == '锌':
                        suggestion_text += " (如贝壳类海产品、红色肉类)"
                    elif micro == '维生素A':
                        suggestion_text += " (如动物肝脏、胡萝卜、深绿色蔬菜)"
                    elif micro == '维生素B1':
                        suggestion_text += " (如全谷物、豆类、瘦猪肉)"
                    elif micro == '维生素B2':
                        suggestion_text += " (如奶制品、动物内脏、蛋类)"
                    elif micro == '维生素C':
                        suggestion_text += " (如新鲜蔬菜水果，特别是深色蔬菜和柑橘类水果)"
                    suggestions.append(suggestion_text)

        # AAS Suggestions (focus on '不合理', '不够合理')
        if aas_eval and aas_eval.get('overall_comment') == '部分餐次 AAS 不足或计算/数据缺失 ❌':
            suggestions.append("部分餐次蛋白质质量（AAS）不理想，需注意食物搭配：")
            for meal, eval_data in aas_eval.get('meal_aas', {}).items():
                aas_comment = eval_data.get('comment')
                if aas_comment in ['不合理', '不够合理']:
                    suggestion_text = f"  {meal}: AAS 不理想 ({eval_data.get('aas', 'N/A'):.1f})，建议在该餐次搭配不同来源的蛋白质食物"
                    if eval_data.get('limiting_aa'):
                        suggestion_text += f" (限制性氨基酸: {eval_data['limiting_aa']})"
                    suggestion_text += "，如谷类和豆类同食，或增加优质动物蛋白。"
                    suggestions.append(suggestion_text)

        # --- Generate Final Report ---
        overall_summary += "\n膳食建议:\n"
        if suggestions:
            for i, suggestion in enumerate(suggestions):
                overall_summary += f"  {i + 1}. {suggestion}\n"
        else:
            overall_summary += "  食谱评价基本符合要求，请继续保持均衡膳食。\n"

        overall_summary += f"\n--- {student_id} 食谱整体评价与建议结束 ---\n"

        # Print the generated summary
        print(overall_summary)

        return overall_summary

    # --- Main Execution Method ---
    def main(self):
        """
        执行食谱的全部评价流程：分析结构、计算营养素、各项评价、生成总结和建议。
        """
        student_id = self.evaluation_results['student_id']
        gender = self.evaluation_results['gender']
        print(f"\n--- 开始评价 {student_id} ({gender}) 的食谱 ---")

        # 1. 分析食物结构
        self.analyze_food_structure()
        # print(f"--- {student_id} 食物结构分析完成 ---") # Printing moved to Step 6 summary

        # 2. 计算主要营养素含量 (获取日总和每餐总)
        df_intake, df_meal = self.calculate_nutrient_intakes()
        # print(f"--- {student_id} 主要营养素含量计算完成 ---") # Printing moved to Step 6 summary

        # 3.1 评价能量
        self.evaluate_energy(df_intake)

        # 3.2 评价餐次比
        self.evaluate_meal_ratio(df_intake, df_meal)

        # 3.3 评价非产能主要营养素
        self.evaluate_micronutrients(df_intake)

        # 4. 评价宏量营养素供能比
        self.evaluate_macro_ratios(df_intake)

        # 5. 评价每餐的蛋白质氨基酸评分 (AAS)
        self.calculate_and_evaluate_per_meal_aas(df_meal)

        # 6. 综合评价和建议 (Prints the final summary)
        self.generate_overall_evaluation_and_suggestions()

        print(f"\n--- {student_id} 食谱评价全部完成 ---")

        # Optional: Return the full evaluation results dictionary
        # return self.evaluation_results


evaluation_instance_man = Evaluation(df_man, display_flag=True, rounding_decimals=2)
evaluation_instance_man.main()

# --- Example Usage (after defining df_man, df_woman, etc.) ---
# Assuming df_man is loaded, cleaned, and df_man.name = '男'

# Create an instance of the Evaluation class
# evaluation_instance_man = Evaluation(df_man, display_flag=True, rounding_decimals=2)

# Run the full evaluation process for the male student
# evaluation_instance_man.main()

# --- Repeat for female student if needed ---
# evaluation_instance_woman = Evaluation(df_woman, display_flag=True, rounding_decimals=2)
# evaluation_instance_woman.main()


--- 开始评价 未知学生 (未知) 的食谱 ---

--- 未知学生 (未知) 食谱整体评价与建议 ---

各项评价总结:
  食物结构:
    种类数量: 11 种 (目标 > 12 种) - 评价: 不足 ❌
    五大类包含: 不齐全 ❌
      缺少的类别: 植物油类
    未归类食物: 豆油, 芝麻油
  能量摄入:
    总能量: 2771.98 kcal - 评价: 标准缺失或数据缺失
  餐次供能比:
    早餐: 29.0% (目标 25%-35%) - 评价: 达标
    午餐: 38.8% (目标 30%-40%) - 评价: 达标
    晚餐: 32.2% (目标 30%-40%) - 评价: 达标
  餐次比整体评价: 达标 ✅
  宏量供能比:
    蛋白质: 15.4% (目标 10%-15%) - 评价: 偏高
    脂肪: 44.1% (目标 20%-30%) - 评价: 偏高
    碳水化合物: 40.5% (目标 50%-65%) - 评价: 偏低
  宏量供能比整体评价: 部分宏量比偏离
  非产能营养素:
    非产能主要营养素评价数据缺失。
  每餐 AAS:
    早餐: AAS 79.1 (限制性氨基酸: 赖氨酸) - 评价: 不够合理
    午餐: AAS 75.4 (限制性氨基酸: 苏氨酸) - 评价: 不够合理
    晚餐: AAS 71.7 (限制性氨基酸: 苏氨酸) - 评价: 不够合理
  每餐 AAS 整体评价: 部分餐次 AAS 不足或计算/数据缺失 ❌

综合评价结论:

膳食建议:
  1. 食谱种类不足，建议增加每日食物种类数量，目标 > 12 种，以增加食物多样性。
  2. 食谱包含的食物类别不全，建议增加 植物油类 等五大类食物的摄入。
  3. 蛋白质 供能比偏高 (15.4%)，建议减少富含 蛋白质 的食物摄入。
  4. 脂肪 供能比偏高 (44.1%)，建议减少富含 脂肪 的食物摄入。
  5. 碳水化合物 供能比偏低 (40.5%)，建议增加富含 碳水化合物 的食物摄入，如全谷物、薯类等。
  6. 部分餐次蛋白质质量（AAS）不理想，需注意食物搭配：
  7.   早餐: AAS 不理想 (79.1)，建议在该餐次搭配不同来源的蛋白质食物 

In [34]:
# -*- coding: utf-8 -*-

import os
import numpy as np
import pandas as pd


# Assuming find_project_root, clean_food_code, clean_food_code_main functions are defined elsewhere and work.
# Assuming df_man_origin, df_woman_origin are loaded.
# Assuming df_man, df_woman are created as copies and have .name set (e.g., df_man.name = '男').


# --- Helper function to get major food group from code (used in Step 1) ---
def get_major_food_group_from_code(food_code, code_to_group_map):
    """
    根据食物编码的前两位查找对应的五大类别名称。
    非五大类别，则返回 '其他类别'。
    Assumes food_code is not NaN and is string-like.
    """
    if pd.isna(food_code) or not isinstance(food_code, str):
        return '其他类别'
    if len(food_code) < 2:
        return '其他类别'
    code = food_code[:2]
    return code_to_group_map.get(code, '其他类别')


# --- Helper function to count unique food types (used in Step 1) ---
def count_food_types(df: pd.DataFrame) -> int:
    """
    统计 DataFrame 中 '食物名称' 列的不重复数量。
    """
    if '食物名称' in df.columns:
        return df['食物名称'].nunique()
    return 0


# --- Evaluation Class (Finalized based on user requirements) ---

class Evaluation:
    def __init__(self, df: pd.DataFrame, gender: str = '未知', display_flag: bool = False, rounding_decimals: int = 2):
        self.df = df.copy()
        self.df.name = gender
        self.display_flag = display_flag
        self.rounding_decimals = rounding_decimals

        # --- Standards (Comprehensive) ---
        self.standards = {
            'daily_variety_min': 12,
            'energy_target': {'男': 2400, '女': 1900},
            'energy_acceptable_range_percent': 10,
            'meal_ratio_range': {'早餐': (0.25, 0.35), '午餐': (0.30, 0.40), '晚餐': (0.30, 0.40)},
            'micro_target': {
                '男': {'钙': 800, '铁': 12, '锌': 12.5, '维生素A': 800, '维生素B1': 1.4, '维生素B2': 1.4,
                       '维生素C': 100},
                '女': {'钙': 800, '铁': 20, '锌': 7.5, '维生素A': 700, '维生素B1': 1.2, '维生素B2': 1.2, '维生素C': 100}
            },
            'macro_ratio_range': {'蛋白质': (0.10, 0.15), '脂肪': (0.20, 0.30), '碳水化合物': (0.50, 0.65)},
            'energy_conversion': {'蛋白质': 4, '脂肪': 9, '碳水化合物': 4},  # No Fiber
            'aas_ref_pattern': {
                '异亮氨酸': 40, '亮氨酸': 70, '赖氨酸': 55, '含硫氨基酸': 35,
                '芳香族氨基酸': 60, '苏氨酸': 40, '色氨酸': 10, '缬氨酸': 50,
            },
            'aas_eval_criteria': {
                '不合理': (0, 60), '不够合理': (60, 80), '比较合理': (80, 90), '合理': (90, float('inf')),
            },
            'five_major_food_names': ["谷、薯类", "蔬菜、菌藻、水果类", "畜、禽、鱼、蛋类及制品", "奶、干豆、坚果、种子类及制品",
                                      "植物油类"],
            'code_prefix_to_major_group': self._build_code_to_group_map(),
            'nutrient_cols_per_100g': self._get_nutrient_cols_per_100g(df),  # Excludes Fiber
            'weight_col': '食物重量(克)',
            'meal_col': '餐次',
        }

        # Initialize results dictionary
        self.evaluation_results = {
            'gender': self.df.name if hasattr(self.df, 'name') and self.df.name in ['男', '女'] else '未知',
            # Use df.name directly for gender
        }

        # Ensure df has .name attribute (and it's valid gender if possible)
        if not hasattr(self.df, 'name') or self.df.name not in ['男', '女']:
            self.df.name = self.evaluation_results['gender']  # Fallback or assign if needed

    def _build_code_to_group_map(self):
        """ 构建食物编码前缀到五大类别的映射字典。"""
        five_major_food_groups_source = [
            {"谷类及制品": "01", "薯类、淀粉及制品": "02"},
            {"蔬菜类及制品": "04", "菌藻类": "05", "水果类及制品": "06"},
            {"畜肉类及制品": "08", "禽肉类及制品": "09", "鱼虾蟹贝类": "12", "蛋类及制品": "11"},
            {"乳类及制品": "10", "干豆类及制品": "03", "坚果、种子类": "07"},
            {"植物油类": "18"}
        ]
        five_major_food_names_source = ["谷、薯类", "蔬菜、菌藻、水果类", "畜、禽、鱼、蛋类及制品",
                                        "奶、干豆、坚果、种子类及制品", "植物油类"]

        code_to_group_map = {}
        for i, group_codes_dict in enumerate(five_major_food_groups_source):
            group_name = five_major_food_names_source[i]
            for sub_group_name, code_prefix in group_codes_dict.items():
                code_to_group_map[code_prefix] = group_name
        return code_to_group_map

    def _get_nutrient_cols_per_100g(self, df: pd.DataFrame) -> list[str]:
        """
        从 DataFrame 的列名中识别出每100g营养成分列。
        不包含膳食纤维。
        """
        nutrient_patterns = {
            '碳水化合物': 'g/100g', '蛋白质': 'g/100g', '脂肪': 'g/100g',
            # '膳食纤维': 'g/100g', # Excluded
            '钙': 'mg/100g', '铁': 'mg/100g', '锌': 'mg/100g',
            '维生素A': 'μg/100g', '维生素B1': 'mg/100g', '维生素B2': 'mg/100g', '维生素C': 'mg/100g',
            '异亮氨酸': 'g/100g', '亮氨酸': 'g/100g', '赖氨酸': 'g/100g',
            '含硫氨基酸': 'g/100g', '芳香族氨基酸': 'g/100g',
            '苏氨酸': 'g/100g', '色氨酸': 'g/100g', '缬氨酸': 'g/100g',
        }

        nutrient_cols = []
        for name, unit_pattern in nutrient_patterns.items():
            col_name = f'{name} ({unit_pattern})'
            if col_name in df.columns:
                nutrient_cols.append(col_name)

        return nutrient_cols

    #  --- 1 分析食物结构 ---
    def analyze_food_structure(self):
        """ 分析食谱中的食物结构，结果存储在 self.evaluation_results['food_structure']。 """
        df = self.df
        standards = self.standards
        gender = self.evaluation_results['gender']  # Use gender

        # print(f"\n--- {gender} - 1 分析食物结构 ---") # Printing moved to Step 6 summary

        if '食物类别' not in df.columns:
            df['食物类别'] = df['食物编码'].apply(
                lambda x: get_major_food_group_from_code(x, standards['code_prefix_to_major_group'])
            )

        categories_present = df[df['食物类别'] != '其他类别']['食物类别'].unique().tolist()
        missing_categories = [cat for cat in standards['five_major_food_names'] if cat not in categories_present]
        all_five_present = len(missing_categories) == 0

        unique_food_types_count = count_food_types(df)
        daily_variety_met = unique_food_types_count > standards['daily_variety_min']

        other_category_items = df[df['食物类别'] == '其他类别']['主要成分'].unique().tolist()

        food_quantities_summary = None
        weight_col = standards.get('weight_col', '食物重量(克)')
        if '食物名称' in df.columns and weight_col in df.columns:
            food_quantities_summary_df = df.groupby('食物名称')[weight_col].sum().reset_index()
            food_quantities_summary = food_quantities_summary_df.to_dict('records')

        self.evaluation_results['food_structure'] = {
            'categories_present': categories_present,
            'missing_categories': missing_categories,
            'all_five_present': all_five_present,
            'unique_food_types_count': unique_food_types_count,
            'daily_variety_met': daily_variety_met,
            'other_category_items': other_category_items,
            'food_quantities_summary': food_quantities_summary,
        }
        # print(f"--- {gender} 食物结构分析完成 ---") # Printing moved to Step 6 summary

    #  --- 2 计算主要营养素含量 ---
    def calculate_nutrient_intakes(self) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        计算日食谱的日总和每餐总营养素摄入量及能量。
        结果存储在 self.evaluation_results['nutrient_intake']。
        不包含膳食纤维的能量计算。

        Returns:
            一个元组，包含：
            - df_intake: DataFrame（一日各营养素摄入量及能量总量，1行）
            - df_meal: DataFrame（餐次的各营养素摄入量及能量总量）
        """
        df = self.df
        gender = self.evaluation_results['gender']  # Use gender

        weight_col = self.standards.get('weight_col', '食物重量(克)')
        meal_col = self.standards.get('meal_col', '餐次')
        energy_conversion = self.standards.get('energy_conversion', {})  # No Fiber
        nutrient_cols_per_100g = self.standards.get('nutrient_cols_per_100g', [])  # No Fiber

        if weight_col not in df.columns:
            df[weight_col] = df['可食部（克/份）'].fillna(0) * df['食用份数'].fillna(0)

        # --- 2.1 计算每行营养素摄入量 ---
        # print(f"\n--- {gender} - 2 计算主要营养素含量 ---") # Printing moved to Step 6 summary
        # print(f"--- 2.1 计算 {gender} 食谱的主要营养素摄入量 ---") # Printing moved to Step 6 summary

        intake_cols = []
        for nutrient_col_100g in nutrient_cols_per_100g:
            parts = nutrient_col_100g.replace(')', '').split('(')
            if len(parts) == 2:
                nutrient_name = parts[0].strip()
                unit_info = parts[1].strip()
                intake_unit = unit_info.split('/')[0].strip()
                intake_col_name = f'{nutrient_name}摄入量 ({intake_unit})'

                if nutrient_col_100g in df.columns and weight_col in df.columns:
                    df[intake_col_name] = (df[weight_col].fillna(0) / 100) * df[nutrient_col_100g].fillna(0)
                    intake_cols.append(intake_col_name)

        # --- Calculate per-row energy contribution from macros (P, F, C) ---
        per_row_energy_contribution_cols = []
        macro_names_for_energy = ['蛋白质', '脂肪', '碳水化合物']  # Based on energy_conversion keys
        for substance in macro_names_for_energy:
            intake_col_g = f'{substance}摄入量 (g)'
            energy_col_name = f'{substance}能量贡献 (kcal)'
            if intake_col_g in df.columns and energy_conversion.get(substance) is not None:
                df[energy_col_name] = df[intake_col_g].fillna(0) * energy_conversion.get(substance, 0)
                per_row_energy_contribution_cols.append(energy_col_name)

        # --- 2.2 计算一日总营养素摄入量 ---
        # print(f"\n--- 2.2 计算 {gender} 一日总营养素摄入量 ---") # Printing moved to Step 6 summary

        existing_intake_cols_for_daily = [col for col in intake_cols if col in df.columns]
        daily_nutrient_intake_dict = {}
        if existing_intake_cols_for_daily:
            for intake_col in existing_intake_cols_for_daily:
                daily_nutrient_intake_dict[intake_col] = df[intake_col].sum()

        # --- 2.3 计算总能量 (日总) ---
        # Calculate total daily energy ONLY from P, F, C intake grams
        protein_total_g = daily_nutrient_intake_dict.get('蛋白质摄入量 (g)', 0)
        fat_total_g = daily_nutrient_intake_dict.get('脂肪摄入量 (g)', 0)
        carb_total_g = daily_nutrient_intake_dict.get('碳水化合物摄入量 (g)', 0)

        total_calculated_energy_kcal = (protein_total_g * energy_conversion.get('蛋白质', 0) +
                                        fat_total_g * energy_conversion.get('脂肪', 0) +
                                        carb_total_g * energy_conversion.get('碳水化合物', 0))

        daily_nutrient_intake_dict['总能量摄入量 (kcal)'] = total_calculated_energy_kcal

        df_intake = pd.DataFrame([daily_nutrient_intake_dict])

        # print(f"--- {gender} 完成计算一日总营养素和能量摄入量 ---") # Printing moved to Step 6 summary

        # --- 2.4 计算每餐总营养素摄入量及能量 ---
        # print(f"--- 2.4 计算 {gender} 每餐总营养素摄入量及能量 ---") # Printing moved to Step 6 summary

        cols_to_sum_per_meal = intake_cols + per_row_energy_contribution_cols

        existing_cols_to_sum_per_meal = [col for col in cols_to_sum_per_meal if col in df.columns]

        if meal_col not in df.columns:
            df_meal = pd.DataFrame()
            # print(f"错误：缺少餐次列 '{meal_col}'，无法计算每餐总量。") # Printing moved to Step 6 summary
        elif df.empty:
            df_meal = pd.DataFrame()
            # print("警告：输入DataFrame为空，无法计算每餐总量。") # Printing moved to Step 6 summary
        elif not existing_cols_to_sum_per_meal:
            df_meal = pd.DataFrame()
            # print("警告：没有找到任何需要按餐次求和的摄入量或能量贡献列。无法计算每餐总量。") # Printing moved to Step 6 summary
        else:
            df_meal = df.groupby(meal_col)[existing_cols_to_sum_per_meal].sum()

            existing_per_meal_energy_cols = [col for col in per_row_energy_contribution_cols if col in df_meal.columns]
            if existing_per_meal_energy_cols:
                df_meal['总能量摄入量 (kcal)'] = df_meal[existing_per_meal_energy_cols].sum(axis=1)
            else:
                df_meal['总能量摄入量 (kcal)'] = 0

        # print(f"--- {gender} 完成计算每餐总营养素和能量摄入量 ---") # Printing moved to Step 6 summary

        self.evaluation_results['nutrient_intake'] = {
            'daily_totals_df': df_intake,
            'meal_totals_df': df_meal
        }

        return df_intake, df_meal

    #  --- 3 评价各项指标 ---

    # --- 3.1 评价能量 ---
    def evaluate_energy(self, df_intake: pd.DataFrame) -> None:
        """
        评价一日总能量摄入量是否符合个体需要 (根据附件4评价原则1)。
        结果存储在 self.evaluation_results['evaluation_energy']。
        """
        gender = self.evaluation_results['gender']
        rounding_decimals = self.rounding_decimals

        energy_evaluation = {}

        actual_energy_kcal = df_intake.get('总能量摄入量 (kcal)', pd.Series([0])).values[0]
        target_energy_kcal = self.standards['energy_target'].get(gender)
        acceptable_range_percent = self.standards.get('energy_acceptable_range_percent')

        energy_evaluation['actual_kcal'] = round(actual_energy_kcal, rounding_decimals)
        energy_evaluation['target_kcal'] = target_energy_kcal

        comment = "标准缺失或数据缺失"
        percentage_of_target = None
        acceptable_range_kcal = None

        if target_energy_kcal is not None and target_energy_kcal > 0:
            percentage_of_target = (actual_energy_kcal / target_energy_kcal) * 100

            if acceptable_range_percent is not None and isinstance(acceptable_range_percent, (int, float)):
                lower_bound = target_energy_kcal * (1 - acceptable_range_percent / 100)
                upper_bound = target_energy_kcal * (1 + acceptable_range_percent / 100)
                acceptable_range_kcal = (round(lower_bound, 2), round(upper_bound, 2))

                if lower_bound <= actual_energy_kcal <= upper_bound:
                    comment = "适宜"
                elif actual_energy_kcal < lower_bound:
                    comment = "偏低"
                else:  # actual_energy_kcal > upper_bound
                    comment = "偏高"
            else:
                comment = "适宜 (范围标准缺失)" if actual_energy_kcal >= target_energy_kcal else "偏低 (范围标准缺失)"

        elif target_energy_kcal == 0:
            comment = "目标为零，无法评价"

        energy_evaluation['percentage_of_target'] = round(percentage_of_target,
                                                          rounding_decimals) if percentage_of_target is not None else None
        energy_evaluation['acceptable_range_kcal'] = acceptable_range_kcal
        energy_evaluation['comment'] = comment

        self.evaluation_results['evaluation_energy'] = energy_evaluation

    # --- 3.2 评价餐次比 ---
    def evaluate_meal_ratio(self, df_intake: pd.DataFrame, df_meal: pd.DataFrame) -> None:
        """
        评价三餐供能比是否在推荐范围内 (根据附件4评价原则2)。
        结果存储在 self.evaluation_results['evaluation_meal_ratio']。
        """
        rounding_decimals = self.rounding_decimals
        meal_ratio_evaluation = {}

        meal_energy_col = '总能量摄入量 (kcal)'
        total_daily_energy = df_intake.get(meal_energy_col, pd.Series([0])).values[0]

        meal_ratio_ranges = self.standards.get('meal_ratio_range')

        meal_ratio_evaluation['meal_ratios'] = {}
        overall_comment = "达标 ✅"

        meal_order = ['早餐', '午餐', '晚餐']

        if total_daily_energy <= 0 or meal_energy_col not in df_meal.columns or df_meal.empty or meal_ratio_ranges is None:
            overall_comment = "总能量为零、每餐能量数据缺失或标准缺失，无法评价"
        else:
            for meal in meal_order:
                meal_energy = df_meal.loc[
                    meal, meal_energy_col] if meal in df_meal.index and meal_energy_col in df_meal.columns else 0
                meal_percentage = (meal_energy / total_daily_energy) if total_daily_energy > 0 else 0
                target_range = meal_ratio_ranges.get(meal)

                meal_eval = {
                    'actual_kcal': round(meal_energy, rounding_decimals),
                    'actual_percentage': round(meal_percentage * 100, rounding_decimals),
                    'target_range_percent': None,
                    'comment': "标准缺失或数据缺失"
                }
                meal_ratio_evaluation['meal_ratios'][meal] = meal_eval

                if target_range is not None and isinstance(target_range, tuple) and len(
                        target_range) == 2 and isinstance(target_range[0], (int, float)) and isinstance(target_range[1],
                                                                                                        (int, float)):
                    target_min_percent = target_range[0] * 100
                    target_max_percent = target_range[1] * 100
                    meal_eval['target_range_percent'] = (round(target_min_percent, 2), round(target_max_percent, 2))

                    if target_range[0] <= meal_percentage <= target_range[1]:
                        meal_eval['comment'] = "达标"
                    else:
                        meal_eval['comment'] = "偏低" if meal_percentage < target_range[0] else "偏高"
                        if overall_comment == "达标 ✅":
                            overall_comment = "部分餐次比偏离"
                else:
                    if overall_comment == "达标 ✅":
                        overall_comment = "部分餐次比标准问题"

        meal_ratio_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_meal_ratio'] = meal_ratio_evaluation

    # --- 3.3 评价非产能主要营养素 ---
    def evaluate_micronutrients(self, df_intake: pd.DataFrame) -> None:
        """
        评价非产能主要营养素摄入量是否达到标准 (根据附件4评价原则3)。
        结果存储在 self.evaluation_results['evaluation_micronutrients']。
        """
        gender = self.evaluation_results['gender']
        rounding_decimals = self.rounding_decimals
        micronutrients_evaluation = {}

        micronutrient_targets = self.standards['micro_target'].get(gender)

        micronutrients_evaluation['micronutrients'] = {}
        overall_comment = "基本达标 😊"

        if micronutrient_targets is None or not isinstance(micronutrient_targets, dict) or not micronutrient_targets:
            overall_comment = "标准缺失，无法评价"
        else:
            micros_to_evaluate = list(micronutrient_targets.keys())

            for micro_name in micros_to_evaluate:
                target_amount = micronutrient_targets.get(micro_name)

                actual_intake_col_name = next(
                    (col for col in df_intake.columns if col.startswith(f'{micro_name}摄入量 (')),
                    None
                )
                actual_intake = df_intake[actual_intake_col_name].values[
                    0] if actual_intake_col_name in df_intake.columns and not df_intake.empty else 0

                micro_eval = {
                    'actual_intake': None,
                    'target_amount': target_amount,
                    'unit': None,
                    'percentage_of_rni_ai': None,
                    'comment': "计算失败或标准缺失"
                }
                micronutrients_evaluation['micronutrients'][micro_name] = micro_eval

                if target_amount is None or not isinstance(target_amount, (int, float)):
                    micro_eval['comment'] = "标准缺失或格式错误"
                    if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素标准问题"
                elif actual_intake_col_name is None:
                    micro_eval['comment'] = "数据列缺失"
                    if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素数据列缺失"
                    overall_comment = "部分未达标，需改进"
                else:
                    micro_eval['actual_intake'] = round(actual_intake, rounding_decimals)
                    actual_unit_match = actual_intake_col_name.split('(')[-1].replace(')',
                                                                                      '').strip() if '(' in actual_intake_col_name else '?'
                    micro_eval['unit'] = actual_unit_match

                    if target_amount > 0:
                        percentage = (actual_intake / target_amount) * 100
                        micro_eval['percentage_of_rni_ai'] = round(percentage, rounding_decimals)

                        if percentage >= 100:
                            micro_eval['comment'] = "达标或偏高"
                        else:
                            micro_eval['comment'] = "不足"
                            if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素摄入不足"
                            overall_comment = "部分未达标，需改进"
                    else:
                        micro_eval['comment'] = "目标为零，无法评价"
                        if overall_comment == "基本达标 😊": overall_comment = "部分非产能营养素标准为零"
                        overall_comment = "部分未达标，需改进"

        micronutrients_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_micronutrients'] = micronutrients_evaluation

    # --- 4 评价宏量营养素供能比 ---
    # Adjusted for no fiber energy
    def evaluate_macro_ratios(self, df_intake: pd.DataFrame) -> None:
        """
        评价一日食谱的宏量营养素供能比是否在推荐范围内 (根据附件4评价原则4)。
        结果存储在 self.evaluation_results['evaluation_macro_ratios']。
        能量计算不包含膳食纤维。
        """
        rounding_decimals = self.rounding_decimals
        macro_ratios_evaluation = {}

        total_daily_energy = df_intake.get('总能量摄入量 (kcal)', pd.Series([0])).values[0]
        protein_g = df_intake.get('蛋白质摄入量 (g)', pd.Series([0])).values[0]
        fat_g = df_intake.get('脂肪摄入量 (g)', pd.Series([0])).values[0]
        carb_g = df_intake.get('碳水化合物摄入量 (g)', pd.Series([0])).values[0]

        macro_ratio_ranges = self.standards.get('macro_ratio_range')
        energy_conversion = self.standards.get('energy_conversion', {})  # No Fiber

        macro_ratios_evaluation['macro_ratios'] = {}
        overall_comment = "达标 ✅"

        if total_daily_energy <= 0 or macro_ratio_ranges is None or not isinstance(macro_ratio_ranges,
                                                                                   dict) or not macro_ratio_ranges or energy_conversion is None or not isinstance(
            energy_conversion, dict) or not energy_conversion:
            overall_comment = "日总能量为零、标准缺失或格式不正确，无法评价"
        else:
            protein_kcal = protein_g * energy_conversion.get('蛋白质', 0)
            fat_kcal = fat_g * energy_conversion.get('脂肪', 0)
            carb_kcal = carb_g * energy_conversion.get('碳水化合物', 0)

            macro_energy_dict = {'蛋白质': protein_kcal, '脂肪': fat_kcal, '碳水化合物': carb_kcal}

            macros_to_evaluate = ['蛋白质', '脂肪', '碳水化合物']
            for macro_name in macros_to_evaluate:
                actual_kcal = macro_energy_dict.get(macro_name, 0)
                actual_ratio = (actual_kcal / total_daily_energy) if total_daily_energy > 0 else 0
                target_range = macro_ratio_ranges.get(macro_name)

                macro_eval = {
                    'actual_kcal': round(actual_kcal, rounding_decimals),
                    'actual_ratio': round(actual_ratio, rounding_decimals),
                    'actual_percentage': round(actual_ratio * 100, rounding_decimals),
                    'target_range': target_range,
                    'target_range_percent': None,
                    'comment': "标准缺失或格式错误"
                }
                macro_ratios_evaluation['macro_ratios'][macro_name] = macro_eval

                if target_range is not None and isinstance(target_range, tuple) and len(
                        target_range) == 2 and isinstance(target_range[0], (int, float)) and isinstance(target_range[1],
                                                                                                        (int, float)):
                    target_min_percent = target_range[0] * 100
                    target_max_percent = target_range[1] * 100
                    macro_eval['target_range_percent'] = (round(target_min_percent, 2), round(target_max_percent, 2))

                    if target_range[0] <= actual_ratio <= target_range[1]:
                        macro_eval['comment'] = "达标"
                    else:
                        macro_eval['comment'] = "偏低" if actual_ratio < target_range[0] else "偏高"
                        if overall_comment == "达标 ✅":
                            overall_comment = "部分宏量比偏离"
                else:
                    if overall_comment == "达标 ✅":
                        overall_comment = "部分宏量比标准问题"

            # Check consistency of P+F+C energy sum vs total daily energy (calculated without fiber)
            sum_pfc_kcal = protein_kcal + fat_kcal + carb_kcal
            if total_daily_energy > 0 and abs(sum_pfc_kcal - total_daily_energy) / total_daily_energy > 0.01:
                if overall_comment == "达标 ✅":
                    overall_comment += " (计算警告: PFC能量总和与日总量不符)"

        macro_ratios_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_macro_ratios'] = macro_ratios_evaluation

    # --- 5 计算并评价每餐的蛋白质氨基酸评分 (AAS) ---
    # Removed print statements
    def calculate_and_evaluate_per_meal_aas(self, df_meal: pd.DataFrame) -> None:
        """
        计算并评价每餐的蛋白质氨基酸评分 (AAS) (根据附件4评价原则5)。
        结果存储在 self.evaluation_results['evaluation_aas']。
        """
        rounding_decimals = self.rounding_decimals
        aas_evaluation = {}

        aas_ref_pattern = self.standards.get('aas_ref_pattern')
        aas_eval_criteria = self.standards.get('aas_eval_criteria')

        aas_evaluation['meal_aas'] = {}
        overall_comment = "达标 ✅"

        if aas_ref_pattern is None or not isinstance(aas_ref_pattern, dict) or not aas_ref_pattern:
            overall_comment = "AAS 参考模式标准缺失或格式不正确，无法评价"
        elif aas_eval_criteria is None or not isinstance(aas_eval_criteria, dict) or not aas_eval_criteria:
            overall_comment = "AAS 评价标准缺失或格式不正确，无法评价"
        elif df_meal.empty:
            overall_comment = "每餐数据为空，无法计算 AAS"
        else:
            protein_intake_col_g = '蛋白质摄入量 (g)'
            essential_aa_names = list(aas_ref_pattern.keys())
            essential_aa_intake_cols = []

            for aa_name in essential_aa_names:
                col_mg = f'{aa_name}摄入量 (mg)'
                col_g = f'{aa_name}摄入量 (g)'
                if col_mg in df_meal.columns:
                    essential_aa_intake_cols.append(col_mg)
                elif col_g in df_meal.columns:
                    essential_aa_intake_cols.append(col_g)

            if protein_intake_col_g not in df_meal.columns:
                overall_comment = f"缺少蛋白质摄入量列 '{protein_intake_col_g}'，无法计算 AAS"
            elif len(essential_aa_intake_cols) < len(essential_aa_names):
                missing_aa_cols = [f'{aa_name}摄入量 (g/mg)' for aa_name in essential_aa_names if
                                   f'{aa_name}摄入量 (mg)' not in essential_aa_intake_cols and f'{aa_name}摄入量 (g)' not in essential_aa_intake_cols]
                overall_comment = f"缺少部分必需氨基酸摄入量列 ({', '.join(missing_aa_cols)})，无法计算 AAS"
            else:
                meal_order = ['早餐', '午餐', '晚餐']
                overall_aas_sufficient = True

                for meal in meal_order:
                    meal_eval_default = {'comment': '数据缺失', 'aas': None, 'limiting_aa': None, 'protein_g': None}

                    if meal not in df_meal.index:
                        aas_evaluation['meal_aas'][meal] = meal_eval_default
                        overall_aas_sufficient = False
                        continue

                    meal_protein_g = df_meal.loc[
                        meal, protein_intake_col_g] if protein_intake_col_g in df_meal.columns else 0

                    if meal_protein_g <= 0:
                        meal_eval_default['comment'] = '蛋白质摄入为零，无法计算 AAS'
                        aas_evaluation['meal_aas'][meal] = meal_eval_default
                        overall_aas_sufficient = False
                        continue

                    limiting_aa = None
                    min_ratio = float('inf')
                    aa_ratios = {}

                    calculation_successful = True

                    for aa_intake_col in essential_aa_intake_cols:
                        aa_name = aa_intake_col.split('摄入量')[0].strip()
                        aa_unit = aa_intake_col.split('(')[-1].replace(')', '').strip()

                        aa_intake_amount = df_meal.loc[meal, aa_intake_col] if aa_intake_col in df_meal.columns else 0

                        aa_intake_mg = aa_intake_amount * 1000 if aa_unit.lower() == 'g' else aa_intake_amount

                        ref_amount_mg_per_g_protein = aas_ref_pattern.get(aa_name)

                        if ref_amount_mg_per_g_protein is None or ref_amount_mg_per_g_protein <= 0:
                            calculation_successful = False
                            break

                        ratio = aa_intake_mg / (meal_protein_g * ref_amount_mg_per_g_protein) if (
                                meal_protein_g > 0 and ref_amount_mg_per_g_protein > 0) else 0
                        aa_ratios[aa_name] = round(ratio, rounding_decimals)

                        if ratio < min_ratio:
                            min_ratio = ratio
                            limiting_aa = aa_name

                    meal_eval = {
                        'protein_g': round(meal_protein_g, rounding_decimals),
                        'aas': None,
                        'limiting_aa': None,
                        'comment': "计算失败",
                    }

                    if calculation_successful and min_ratio != float('inf'):
                        aas_score = round(min_ratio * 100, rounding_decimals)
                        meal_eval['aas'] = aas_score
                        meal_eval['limiting_aa'] = limiting_aa

                        meal_aas_comment = "未知等级"
                        is_current_meal_sufficient = False
                        sorted_criteria = sorted(aas_eval_criteria.items(), key=lambda item: item[1][0])

                        for comment, range_tuple in sorted_criteria:
                            if range_tuple[0] <= aas_score < range_tuple[1]:
                                meal_aas_comment = comment
                                if comment in ['合理', '比较合理']:
                                    is_current_meal_sufficient = True
                                break

                        meal_eval['comment'] = meal_aas_comment

                        if not is_current_meal_sufficient:
                            overall_aas_sufficient = False

                    else:
                        overall_aas_sufficient = False

                    aas_evaluation['meal_aas'][meal] = meal_eval

                if overall_aas_sufficient:
                    overall_comment = "所有餐次 AAS 达标 ✅"
                else:
                    overall_comment = "部分餐次 AAS 不足或计算/数据缺失 ❌"

        aas_evaluation['overall_comment'] = overall_comment
        self.evaluation_results['evaluation_aas'] = aas_evaluation

    # --- 6 综合所有评价结果，生成整体评价和膳食建议 ---
    def generate_overall_evaluation_and_suggestions(self) -> str:
        """
        综合所有评价结果，生成整体评价和膳食建议。
        """
        evaluation_results = self.evaluation_results
        standards = self.standards
        gender = evaluation_results.get('gender', '未知')  # Use gender
        rounding_decimals = self.rounding_decimals

        # Use gender in the main title
        overall_summary = f"\n--- {gender} 学生食谱整体评价与建议 ---\n"

        overall_summary += "\n各项评价总结:\n"

        # 1. 食物结构总结
        print('-' * 50)
        fs_eval = evaluation_results.get('food_structure', {})
        overall_summary += "  食物结构:\n"
        if fs_eval:
            overall_summary += f"    种类数量: {fs_eval.get('unique_food_types_count', 'N/A')} 种 (目标 > {standards.get('daily_variety_min', 'N/A')} 种) - 评价: {'达标 ✅' if fs_eval.get('daily_variety_met', False) else '不足 ❌'}\n"
            overall_summary += f"    五大类包含: {'齐全 ✅' if fs_eval.get('all_five_present', False) else '不齐全 ❌'}\n"
            if fs_eval.get('missing_categories'):
                overall_summary += f"      缺少的类别: {', '.join(fs_eval['missing_categories'])}\n"
            if fs_eval.get('other_category_items'):
                overall_summary += f"    未归类食物: {', '.join(fs_eval['other_category_items'])}\n"
        else:
            overall_summary += "    食物结构评价数据缺失。\n"

        # 3.1 能量摄入总结
        energy_eval = evaluation_results.get('evaluation_energy', {})
        overall_summary += "  能量摄入:\n"
        if energy_eval:
            actual_energy_kcal = energy_eval.get('actual_kcal', 'N/A')
            target_energy_kcal = energy_eval.get('target_kcal')
            percentage = energy_eval.get('percentage_of_target', 'N/A')
            acceptable_range_kcal = energy_eval.get('acceptable_range_kcal', 'N/A')

            summary_line = f"    总能量: {actual_energy_kcal:.{rounding_decimals}f} kcal"
            if target_energy_kcal is not None:
                summary_line += f" (目标 {target_energy_kcal:.0f} kcal"
                if acceptable_range_kcal != 'N/A':
                    summary_line += f", 适宜范围 {acceptable_range_kcal[0]:.0f}-{acceptable_range_kcal[1]:.0f} kcal"
                summary_line += f", 占目标 {percentage:.1f}%)"
            summary_line += f" - 评价: {energy_eval.get('comment', 'N/A')}\n"
            overall_summary += summary_line
        else:
            overall_summary += "    能量评价数据缺失。\n"

        # 3.2 餐次比总结
        meal_ratio_eval = evaluation_results.get('evaluation_meal_ratio', {})
        overall_summary += "  餐次供能比:\n"
        if meal_ratio_eval and meal_ratio_eval.get('meal_ratios'):
            for meal, eval_data in meal_ratio_eval['meal_ratios'].items():
                summary_line = f"    {meal}: {eval_data.get('actual_percentage', 'N/A'):.1f}%"
                if eval_data.get('target_range_percent'):
                    summary_line += f" (目标 {eval_data['target_range_percent'][0]:.0f}%-{eval_data['target_range_percent'][1]:.0f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  餐次比整体评价: {meal_ratio_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    餐次比评价数据缺失。\n"

        # 4. 宏量营养素供能比总结

        macro_eval = evaluation_results.get('evaluation_macro_ratios', {})
        overall_summary += "  宏量供能比:\n"
        if macro_eval and macro_eval.get('macro_ratios'):
            for macro, eval_data in macro_eval['macro_ratios'].items():
                summary_line = f"    {macro}: {eval_data.get('actual_percentage', 'N/A'):.1f}%"
                if eval_data.get('target_range_percent'):
                    summary_line += f" (目标 {eval_data['target_range_percent'][0]:.0f}%-{eval_data['target_range_percent'][1]:.0f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  宏量供能比整体评价: {macro_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    宏量营养素供能比评价数据缺失。\n"

        # 3.3 非产能主要营养素总结
        micro_eval = evaluation_results.get('evaluation_micronutrients', {})
        overall_summary += "  非产能营养素:\n"
        if micro_eval and micro_eval.get('micronutrients'):
            for micro, eval_data in micro_eval['micronutrients'].items():
                summary_line = f"    {micro}: {eval_data.get('actual_intake', 'N/A'):.{rounding_decimals}f}{eval_data.get('unit', '?')}"
                if eval_data.get('target_amount') is not None:
                    summary_line += f" (目标 {eval_data['target_amount']:.0f}{eval_data.get('unit', '?')}, 占目标 {eval_data.get('percentage_of_rni_ai', 'N/A'):.1f}%)"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  非产能营养素整体评价: {micro_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    非产能主要营养素评价数据缺失。\n"

        # 5. AAS 总结
        aas_eval = evaluation_results.get('evaluation_aas', {})
        overall_summary += "  每餐 AAS:\n"
        if aas_eval and aas_eval.get('meal_aas'):
            for meal, eval_data in aas_eval['meal_aas'].items():
                summary_line = f"    {meal}: AAS {eval_data.get('aas', 'N/A'):.1f}"
                if eval_data.get('limiting_aa'):
                    summary_line += f" (限制性氨基酸: {eval_data['limiting_aa']})"
                summary_line += f" - 评价: {eval_data.get('comment', 'N/A')}\n"
                overall_summary += summary_line
            overall_summary += f"  每餐 AAS 整体评价: {aas_eval.get('overall_comment', 'N/A')}\n"
        else:
            overall_summary += "    每餐 AAS 评价数据缺失或计算失败。\n"

        # --- Comprehensive Evaluation Conclusions and Dietary Suggestions ---
        overall_summary += "\n综合评价结论:\n"
        suggestions = []

        # 食物种类建议
        if fs_eval and not fs_eval.get('daily_variety_met', False):
            suggestions.append(
                f"食谱种类不足，建议增加每日食物种类数量，目标 > {standards.get('daily_variety_min', 'N/A')} 种，以增加食物多样性。")
        if fs_eval and not fs_eval.get('all_five_present', False):
            suggestions.append(
                f"食谱包含的食物类别不全，建议增加 {', '.join(fs_eval.get('missing_categories', []))} 等五大类食物的摄入。")

        # 能量建议
        energy_comment = energy_eval.get('comment')
        if energy_comment == '偏低':
            suggestions.append(
                f"总能量摄入偏低 ({energy_eval.get('actual_kcal', 'N/A'):.0f} kcal)，建议适量增加食物摄入总量。")
        elif energy_comment == '偏高':
            suggestions.append(
                f"总能量摄入偏高 ({energy_eval.get('actual_kcal', 'N/A'):.0f} kcal)，建议适量减少食物摄入总量。")

        # 非产能营养素建议
        if meal_ratio_eval and meal_ratio_eval.get('overall_comment') == '部分餐次比偏离':
            for meal, eval_data in meal_ratio_eval.get('meal_ratios', {}).items():
                meal_comment = eval_data.get('comment')
                if meal_comment == '偏低':
                    suggestions.append(
                        f"{meal} 供能比偏低 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议增加 {meal} 的食物摄入量，以使供能更均衡。")
                elif meal_comment == '偏高':
                    suggestions.append(
                        f"{meal} 供能比偏高 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议减少 {meal} 的食物摄入量，并合理分配到其他餐次。")

        # 供能建议
        if macro_eval and macro_eval.get('overall_comment') == '部分宏量比偏离':
            for macro, eval_data in macro_eval.get('macro_ratios', {}).items():
                macro_comment = eval_data.get('comment')
                if macro_comment == '偏低':
                    suggestions.append(
                        f"{macro} 供能比偏低 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议增加富含 {macro} 的食物摄入，如{'全谷物、薯类' if macro == '碳水化合物' else ('鱼禽蛋瘦肉、豆制品' if macro == '蛋白质' else '优质植物油、坚果')}等。")
                elif macro_comment == '偏高':
                    suggestions.append(
                        f"{macro} 供能比偏高 ({eval_data.get('actual_percentage', 'N/A'):.1f}%)，建议减少富含 {macro} 的食物摄入。")

        # Micronutrient Suggestions (focus on '不足')
        if micro_eval and micro_eval.get('overall_comment') in ['部分非产能营养素摄入不足', '部分未达标，需改进']:
            suggestions.append("食谱中部分非产能主要营养素摄入不足，需重点改进：")
            for micro, eval_data in micro_eval.get('micronutrients', {}).items():
                micro_comment = eval_data.get('comment')
                if micro_comment in ['不足', '严重不足']:
                    suggestion_text = f"  {micro}: 摄入量不足 (占目标 {eval_data.get('percentage_of_rni_ai', 'N/A'):.1f}%)，建议增加富含 {micro} 的食物。"
                    if micro == '钙':
                        suggestion_text += " (如奶制品、豆制品、深绿色蔬菜)"
                    elif micro == '铁':
                        suggestion_text += " (如瘦肉、动物肝脏、木耳)"
                    elif micro == '锌':
                        suggestion_text += " (如贝壳类海产品、红色肉类)"
                    elif micro == '维生素A':
                        suggestion_text += " (如动物肝脏、胡萝卜、深绿色蔬菜)"
                    elif micro == '维生素B1':
                        suggestion_text += " (如全谷物、豆类、瘦猪肉)"
                    elif micro == '维生素B2':
                        suggestion_text += " (如奶制品、动物内脏、蛋类)"
                    elif micro == '维生素C':
                        suggestion_text += " (如新鲜蔬菜水果，特别是深色蔬菜和柑橘类水果)"
                    suggestions.append(suggestion_text)

        # AAS Suggestions (focus on '不合理', '不够合理')
        if aas_eval and aas_eval.get('overall_comment') == '部分餐次 AAS 不足或计算/数据缺失 ❌':
            suggestions.append("部分餐次蛋白质质量（AAS）不理想，需注意食物搭配：")
            for meal, eval_data in aas_eval.get('meal_aas', {}).items():
                aas_comment = eval_data.get('comment')
                if aas_comment in ['不合理', '不够合理']:
                    suggestion_text = f"  {meal}: AAS 不理想 ({eval_data.get('aas', 'N/A'):.1f})，建议在该餐次搭配不同来源的蛋白质食物"
                    if eval_data.get('limiting_aa'):
                        suggestion_text += f" (限制性氨基酸: {eval_data['limiting_aa']})"
                    suggestion_text += "，如谷类和豆类同食，或增加优质动物蛋白。"
                    suggestions.append(suggestion_text)

        # --- Generate Final Report ---
        overall_summary += "\n膳食建议:\n"
        if suggestions:
            for i, suggestion in enumerate(suggestions):
                overall_summary += f"  {i + 1}. {suggestion}\n"
        else:
            overall_summary += "  食谱评价基本符合要求，请继续保持均衡膳食。\n"

        # Use gender in the footer
        overall_summary += f"\n--- {gender} 学生食谱整体评价与建议结束 ---\n"

        print(overall_summary)

        return overall_summary

    # --- Main Execution Method ---
    def main(self):
        """
        执行食谱的全部评价流程：分析结构、计算营养素、各项评价、生成总结和建议。
        不使用 student_id，只使用 gender。
        """
        gender = self.evaluation_results['gender']  # Use gender

        # Use gender in the start message
        print(f"\n--- 开始评价 {gender} 学生 的食谱 ---")

        # 1. 分析食物结构
        self.analyze_food_structure()

        # 2. 计算主要营养素含量 (获取日总和每餐总)
        df_intake, df_meal = self.calculate_nutrient_intakes()

        # 3.1 评价能量
        self.evaluate_energy(df_intake)

        # 3.2 评价餐次比
        self.evaluate_meal_ratio(df_intake, df_meal)

        # 3.3 评价非产能主要营养素
        self.evaluate_micronutrients(df_intake)

        # 4. 评价宏量营养素供能比
        self.evaluate_macro_ratios(df_intake)

        # 5. 评价每餐的蛋白质氨基酸评分 (AAS)
        self.calculate_and_evaluate_per_meal_aas(df_meal)

        # 6. 综合评价和建议 (Prints the final summary)
        self.generate_overall_evaluation_and_suggestions()

        # Use gender in the end message
        print(f"\n--- {gender} 学生 食谱评价全部完成 ---")

        # Optional: Return the full evaluation results dictionary
        # return self.evaluation_results


--- 开始评价 男 学生 的食谱 ---

--- 男 学生食谱整体评价与建议 ---

各项评价总结:
  食物结构:
    种类数量: 11 种 (目标 > 12 种) - 评价: 不足 ❌
    五大类包含: 不齐全 ❌
      缺少的类别: 植物油类
    未归类食物: 豆油, 芝麻油
  能量摄入:
    总能量: 2771.98 kcal (目标 2400 kcal, 适宜范围 2160-2640 kcal, 占目标 115.5%) - 评价: 偏高
  餐次供能比:
    早餐: 29.0% (目标 25%-35%) - 评价: 达标
    午餐: 38.8% (目标 30%-40%) - 评价: 达标
    晚餐: 32.2% (目标 30%-40%) - 评价: 达标
  餐次比整体评价: 达标 ✅
  宏量供能比:
    蛋白质: 15.4% (目标 10%-15%) - 评价: 偏高
    脂肪: 44.1% (目标 20%-30%) - 评价: 偏高
    碳水化合物: 40.5% (目标 50%-65%) - 评价: 偏低
  宏量供能比整体评价: 部分宏量比偏离
  非产能营养素:
    钙: 412.50mg (目标 800mg, 占目标 51.6%) - 评价: 不足
    铁: 18.55mg (目标 12mg, 占目标 154.6%) - 评价: 达标或偏高
    锌: 11.34mg (目标 12mg, 占目标 90.7%) - 评价: 不足
    维生素A: 107.95μg (目标 800μg, 占目标 13.5%) - 评价: 不足
    维生素B1: 2.16mg (目标 1mg, 占目标 154.0%) - 评价: 达标或偏高
    维生素B2: 1.52mg (目标 1mg, 占目标 108.5%) - 评价: 达标或偏高
    维生素C: 42.16mg (目标 100mg, 占目标 42.2%) - 评价: 不足
  非产能营养素整体评价: 部分未达标，需改进
  每餐 AAS:
    早餐: AAS 79.1 (限制性氨基酸: 赖氨酸) - 评价: 不够合理
    午餐: AAS 75.4 (限制性氨基酸: 苏氨酸) - 评价: 不够合理
    晚餐: AAS 7

In [32]:
df_man.name

'男'